In [1]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 1: Imports and Data Structures

import asyncio
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any, Tuple
import warnings
import os
import json
import re
warnings.filterwarnings('ignore')

# Optional imports with proper error handling
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("⚠️ Prophet not available - using simple forecasting")

try:
    from openai import OpenAI
    HAS_OPENAI = True
except ImportError:
    HAS_OPENAI = False
    print("⚠️ OpenAI not available - using rule-based recommendations")

try:
    import streamlit as st
    HAS_STREAMLIT = True
except ImportError:
    HAS_STREAMLIT = False
    print("⚠️ Streamlit not available for web app")

# =============================================================================
# DATA STRUCTURES
# =============================================================================

@dataclass
class MarketData:
    symbol: str
    current_price: float
    prices: pd.Series
    volume: pd.Series = field(default_factory=pd.Series)

    # Technical indicators
    rsi: float = 50.0
    trend: str = "neutral"
    return_1d: float = 0.0
    return_5d: float = 0.0
    return_20d: float = 0.0
    volatility_20d: float = 0.0
    support_level: float = 0.0
    resistance_level: float = 0.0

    # Additional technical indicators
    macd_signal: str = "neutral"
    bollinger_position: str = "middle"
    volume_trend: str = "neutral"

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class ForecastData:
    arima_forecast: float = 0.0
    prophet_forecast: float = 0.0
    lstm_forecast: float = 0.0
    ensemble_forecast: float = 0.0
    forecast_confidence: float = 0.5
    prediction_interval: List[float] = field(default_factory=lambda: [0.0, 0.0])
    forecast_horizon_days: int = 5
    forecast_accuracy_score: float = 0.0

    # Enhanced forecast metrics
    upside_probability: float = 0.5
    downside_risk: float = 0.5
    volatility_forecast: float = 0.2
    simulated_returns: List[float] = field(default_factory=list)

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class MacroData:
    gdp_growth: float = 0.0
    inflation_rate: float = 0.0
    unemployment_rate: float = 0.0
    federal_funds_rate: float = 0.0
    vix: float = 0.0
    dollar_index: float = 0.0
    market_sentiment: str = "neutral"

    # Additional macro indicators
    yield_curve_slope: float = 0.0
    credit_spreads: float = 0.0
    economic_surprise_index: float = 0.0

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class SentimentData:
    news_sentiment: float = 0.0
    social_media_sentiment: float = 0.0
    overall_sentiment: float = 0.0
    sentiment_trend: str = "neutral"
    confidence_score: float = 0.5
    key_topics: List[str] = field(default_factory=list)

    # Enhanced sentiment metrics
    sentiment_momentum: float = 0.0
    fear_greed_index: float = 50.0
    analyst_rating_trend: str = "neutral"

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class RiskMetrics:
    """Risk analysis results from RiskAgent"""
    portfolio_volatility: float = 0.0
    value_at_risk_5pct: float = 0.0
    value_at_risk_1pct: float = 0.0
    expected_shortfall: float = 0.0
    maximum_drawdown: float = 0.0
    sharpe_ratio: float = 0.0
    sortino_ratio: float = 0.0
    calmar_ratio: float = 0.0

    # GARCH results
    garch_volatility: float = 0.0
    garch_forecast: List[float] = field(default_factory=list)

    # Rolling metrics
    rolling_volatility: pd.Series = field(default_factory=pd.Series)
    rolling_sharpe: pd.Series = field(default_factory=pd.Series)

    # Drawdown analysis
    drawdown_series: pd.Series = field(default_factory=pd.Series)
    drawdown_periods: List[Dict] = field(default_factory=list)

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class PersonalizedRecommendation:
    """Enhanced recommendation with GPT reasoning"""
    action: str  # "BUY", "SELL", "HOLD"
    confidence: float
    position_size: float
    entry_price: float
    stop_loss: float
    take_profit: float
    risk_level: str
    time_horizon: str

    # GPT-enhanced fields
    detailed_reasoning: str
    key_risk_factors: List[str]
    key_opportunity_factors: List[str]
    alternative_scenarios: Dict[str, str]
    portfolio_impact: str
    market_timing_analysis: str

    # Quantitative metrics
    risk_reward_ratio: float
    probability_of_success: float
    maximum_drawdown_estimate: float

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class FinancialGoal:
    """Enhanced financial goal with more parameters"""
    target_amount: float
    current_amount: float
    monthly_contribution: float
    time_horizon_years: int
    risk_tolerance: str  # "conservative", "moderate", "aggressive"
    age: int = 30
    annual_income: float = 100000

    # Enhanced parameters
    goal_type: str = "retirement"  # "retirement", "house", "education", "general"
    existing_debt: float = 0.0
    emergency_fund: float = 0.0
    other_investments: float = 0.0
    tax_rate: float = 0.22
    inflation_assumption: float = 0.03

@dataclass
class FinancialPlanResult:
    """Comprehensive financial plan results"""
    goal: FinancialGoal
    projected_value: float
    success_probability: float
    required_monthly: float
    asset_allocation: Dict[str, float]
    tax_optimization: Dict[str, float]
    monthly_breakdown: Dict[str, float]
    recommendations: List[str]
    is_achievable: bool
    monte_carlo_results: Dict[str, float]

    # Risk-adjusted metrics
    plan_sharpe_ratio: float = 0.0
    plan_max_drawdown: float = 0.0
    plan_volatility: float = 0.0

    last_updated: datetime = field(default_factory=datetime.now)

⚠️ Streamlit not available for web app


In [2]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 2: Agent 1 - Market Data Agent

# =============================================================================
# AGENT 1: MARKET DATA AGENT
# =============================================================================

class MarketDataAgent:
    """Enhanced market data agent with technical indicators"""

    def __init__(self):
        self.name = "MarketDataAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict, symbol: str = "AAPL", period: str = "1y") -> Dict:
        """Fetch and process market data with technical analysis"""

        try:
            print(f"📊 {self.name}: Fetching market data for {symbol}...")

            # Fetch data from Yahoo Finance
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            if data.empty:
                raise ValueError(f"No data available for {symbol}")

            # Calculate enhanced metrics
            current_price = float(data['Close'].iloc[-1])
            prices = data['Close']
            volume = data['Volume']

            # Basic returns and volatility
            returns = prices.pct_change()
            return_1d = float(returns.iloc[-1]) if len(returns) > 0 else 0.0
            return_5d = float(returns.tail(5).mean()) if len(returns) >= 5 else 0.0
            return_20d = float(returns.tail(20).mean()) if len(returns) >= 20 else 0.0
            volatility_20d = float(returns.tail(20).std() * np.sqrt(252)) if len(returns) >= 20 else 0.0

            # Enhanced technical indicators
            rsi = self._calculate_rsi(prices)
            trend = self._analyze_trend(prices)
            macd_signal = self._calculate_macd_signal(prices)
            bollinger_position = self._calculate_bollinger_position(prices)
            volume_trend = self._analyze_volume_trend(volume)

            # Support and resistance levels
            high_20 = prices.tail(20).max()
            low_20 = prices.tail(20).min()
            support_level = float(low_20 * 1.02)
            resistance_level = float(high_20 * 0.98)

            # Create market data object
            market_data = MarketData(
                symbol=symbol,
                current_price=current_price,
                prices=prices,
                volume=volume,
                rsi=rsi,
                trend=trend,
                return_1d=return_1d,
                return_5d=return_5d,
                return_20d=return_20d,
                volatility_20d=volatility_20d,
                support_level=support_level,
                resistance_level=resistance_level,
                macd_signal=macd_signal,
                bollinger_position=bollinger_position,
                volume_trend=volume_trend
            )

            # Store in state
            state['market_data'] = market_data
            state['symbol'] = symbol

            print(f"✅ {self.name}: Loaded {len(prices)} data points")
            print(f"   Price: ${current_price:.2f}")
            print(f"   Trend: {trend}")
            print(f"   RSI: {rsi:.1f}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _calculate_rsi(self, prices: pd.Series, period: int = 14) -> float:
        """Calculate RSI indicator"""
        if len(prices) < period + 1:
            return 50.0

        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return float(rsi.iloc[-1]) if not np.isnan(rsi.iloc[-1]) else 50.0

    def _analyze_trend(self, prices: pd.Series) -> str:
        """Analyze price trend using multiple moving averages"""
        if len(prices) < 20:
            return "neutral"

        ma_5 = prices.tail(5).mean()
        ma_10 = prices.tail(10).mean()
        ma_20 = prices.tail(20).mean()

        if ma_5 > ma_10 > ma_20:
            return "strongly_bullish"
        elif ma_5 > ma_20 * 1.02:
            return "bullish"
        elif ma_5 < ma_10 < ma_20:
            return "strongly_bearish"
        elif ma_5 < ma_20 * 0.98:
            return "bearish"
        else:
            return "neutral"

    def _calculate_macd_signal(self, prices: pd.Series) -> str:
        """Calculate MACD signal"""
        if len(prices) < 26:
            return "neutral"

        exp1 = prices.ewm(span=12).mean()
        exp2 = prices.ewm(span=26).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9).mean()

        if macd.iloc[-1] > signal.iloc[-1]:
            return "bullish"
        elif macd.iloc[-1] < signal.iloc[-1]:
            return "bearish"
        else:
            return "neutral"

    def _calculate_bollinger_position(self, prices: pd.Series, period: int = 20) -> str:
        """Calculate position relative to Bollinger Bands"""
        if len(prices) < period:
            return "middle"

        sma = prices.rolling(window=period).mean()
        std = prices.rolling(window=period).std()

        upper_band = sma + (std * 2)
        lower_band = sma - (std * 2)

        current_price = prices.iloc[-1]
        current_upper = upper_band.iloc[-1]
        current_lower = lower_band.iloc[-1]
        current_middle = sma.iloc[-1]

        if current_price > current_upper:
            return "above_upper"
        elif current_price < current_lower:
            return "below_lower"
        elif current_price > current_middle:
            return "upper_half"
        else:
            return "lower_half"

    def _analyze_volume_trend(self, volume: pd.Series) -> str:
        """Analyze volume trend"""
        if len(volume) < 10:
            return "neutral"

        recent_volume = volume.tail(5).mean()
        historical_volume = volume.tail(20).mean()

        if recent_volume > historical_volume * 1.2:
            return "increasing"
        elif recent_volume < historical_volume * 0.8:
            return "decreasing"
        else:
            return "stable"

In [3]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 3: Agent 2 - Risk Agent (NEW - CAPSTONE REQUIREMENT)

# =============================================================================
# AGENT 2: RISK AGENT (CAPSTONE REQUIREMENT #1)
# =============================================================================

class RiskAgent:
    """Risk analysis agent with GARCH and portfolio risk metrics"""

    def __init__(self):
        self.name = "RiskAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict) -> Dict:
        """Calculate comprehensive risk metrics including GARCH and Sharpe ratio"""

        try:
            print(f"📊 {self.name}: Computing portfolio risk metrics...")

            if 'market_data' not in state or not state['market_data']:
                print(f"⚠️ {self.name}: No market data available")
                return state

            market_data = state['market_data']
            prices = market_data.prices

            if len(prices) < 30:
                print(f"⚠️ {self.name}: Insufficient data for risk analysis")
                return state

            # Calculate returns
            returns = prices.pct_change().dropna()

            # Basic risk metrics
            portfolio_volatility = float(returns.std() * np.sqrt(252))  # Annualized

            # Value at Risk (Historical method)
            var_5pct = float(np.percentile(returns, 5))
            var_1pct = float(np.percentile(returns, 1))

            # Expected Shortfall (Conditional VaR)
            expected_shortfall = float(returns[returns <= var_5pct].mean())

            # Maximum Drawdown calculation
            cumulative_returns = (1 + returns).cumprod()
            running_max = cumulative_returns.expanding().max()
            drawdown = (cumulative_returns - running_max) / running_max
            maximum_drawdown = float(drawdown.min())

            # Sharpe Ratio (CAPSTONE REQUIREMENT #4)
            risk_free_rate = 0.02  # 2% risk-free rate
            excess_returns = returns.mean() * 252 - risk_free_rate
            sharpe_ratio = float(excess_returns / portfolio_volatility) if portfolio_volatility > 0 else 0.0

            # Sortino Ratio (downside deviation focus)
            downside_returns = returns[returns < 0]
            downside_deviation = float(downside_returns.std() * np.sqrt(252)) if len(downside_returns) > 0 else portfolio_volatility
            sortino_ratio = float(excess_returns / downside_deviation) if downside_deviation > 0 else 0.0

            # Calmar Ratio (return vs max drawdown)
            annual_return = float(returns.mean() * 252)
            calmar_ratio = float(annual_return / abs(maximum_drawdown)) if maximum_drawdown != 0 else 0.0

            # GARCH volatility (CAPSTONE REQUIREMENT #1)
            garch_volatility = self._calculate_garch_volatility(returns)
            garch_forecast = self._forecast_garch_volatility(returns, horizon=5)

            # Rolling metrics (CAPSTONE REQUIREMENT #1)
            rolling_vol = returns.rolling(window=20).std() * np.sqrt(252)
            rolling_sharpe = self._calculate_rolling_sharpe(returns, window=60)

            # Detailed drawdown analysis (CAPSTONE REQUIREMENT #4)
            drawdown_periods = self._analyze_drawdown_periods(drawdown)

            # Create risk metrics object
            risk_metrics = RiskMetrics(
                portfolio_volatility=portfolio_volatility,
                value_at_risk_5pct=var_5pct,
                value_at_risk_1pct=var_1pct,
                expected_shortfall=expected_shortfall,
                maximum_drawdown=maximum_drawdown,
                sharpe_ratio=sharpe_ratio,
                sortino_ratio=sortino_ratio,
                calmar_ratio=calmar_ratio,
                garch_volatility=garch_volatility,
                garch_forecast=garch_forecast,
                rolling_volatility=rolling_vol,
                rolling_sharpe=rolling_sharpe,
                drawdown_series=drawdown,
                drawdown_periods=drawdown_periods
            )

            state['risk_metrics'] = risk_metrics

            print(f"✅ {self.name}: Risk analysis complete")
            print(f"   Volatility: {portfolio_volatility:.1%}")
            print(f"   Sharpe Ratio: {sharpe_ratio:.2f}")
            print(f"   Max Drawdown: {maximum_drawdown:.1%}")
            print(f"   VaR (5%): {var_5pct:.1%}")
            print(f"   GARCH Vol: {garch_volatility:.1%}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _calculate_garch_volatility(self, returns: pd.Series) -> float:
        """Simplified GARCH(1,1) volatility calculation"""
        try:
            # Simple EWMA as GARCH approximation
            lambda_param = 0.94  # Decay factor (typical for daily data)

            # Calculate exponentially weighted variance
            squared_returns = returns ** 2
            ewma_variance = squared_returns.ewm(alpha=1-lambda_param).mean()

            # Return annualized volatility
            return float(np.sqrt(ewma_variance.iloc[-1] * 252))

        except Exception:
            # Fallback to rolling volatility
            return float(returns.rolling(window=20).std().iloc[-1] * np.sqrt(252))

    def _forecast_garch_volatility(self, returns: pd.Series, horizon: int = 5) -> List[float]:
        """Forecast GARCH volatility for next 'horizon' periods"""
        try:
            current_vol = self._calculate_garch_volatility(returns)
            long_term_vol = float(returns.std() * np.sqrt(252))

            # Simple mean reversion forecast
            forecasts = []
            decay_rate = 0.05  # Mean reversion speed

            for i in range(horizon):
                # Mean revert to long-term volatility
                forecast_vol = current_vol * (1 - decay_rate * i) + long_term_vol * (decay_rate * i)
                forecasts.append(forecast_vol)

            return forecasts

        except Exception:
            # Fallback to constant volatility
            vol = float(returns.std() * np.sqrt(252))
            return [vol] * horizon

    def _calculate_rolling_sharpe(self, returns: pd.Series, window: int = 60) -> pd.Series:
        """Calculate rolling Sharpe ratio"""
        try:
            risk_free_rate = 0.02 / 252  # Daily risk-free rate

            rolling_mean = returns.rolling(window=window).mean()
            rolling_std = returns.rolling(window=window).std()

            rolling_sharpe = (rolling_mean - risk_free_rate) / rolling_std * np.sqrt(252)

            return rolling_sharpe

        except Exception:
            return pd.Series(index=returns.index, data=0.0)

    def _analyze_drawdown_periods(self, drawdown: pd.Series) -> List[Dict]:
        """Analyze individual drawdown periods"""
        try:
            drawdown_periods = []

            # Find drawdown periods (when drawdown < -0.01, i.e., > 1% loss)
            in_drawdown = drawdown < -0.01

            if not in_drawdown.any():
                return drawdown_periods

            # Find start and end of drawdown periods
            drawdown_changes = in_drawdown.diff()
            starts = drawdown_changes[drawdown_changes == True].index
            ends = drawdown_changes[drawdown_changes == False].index

            # Handle edge cases
            if in_drawdown.iloc[0]:
                starts = [drawdown.index[0]] + list(starts)
            if in_drawdown.iloc[-1]:
                ends = list(ends) + [drawdown.index[-1]]

            # Analyze each period
            for start, end in zip(starts, ends):
                period_drawdown = drawdown.loc[start:end]
                max_dd = period_drawdown.min()
                duration = len(period_drawdown)

                drawdown_periods.append({
                    'start_date': start,
                    'end_date': end,
                    'duration_days': duration,
                    'max_drawdown': float(max_dd),
                    'recovery_time': 0  # Could be calculated with more data
                })

            return drawdown_periods[:5]  # Return top 5 worst periods

        except Exception:
            return []

In [5]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 4: Agent 3 - Forecasting Agent

# =============================================================================
# AGENT 3: FORECASTING AGENT
# =============================================================================

class ForecastingAgent:
    """Enhanced forecasting agent with simulated returns for risk analysis"""

    def __init__(self):
        self.name = "ForecastingAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict, forecast_horizon: int = 5) -> Dict:
        """Generate price forecasts with simulated return sequences"""

        if 'market_data' not in state or not state['market_data']:
            print(f"⚠️ {self.name}: No market data available")
            return state

        try:
            print(f"🔮 {self.name}: Generating forecasts...")

            market_data = state['market_data']
            prices = market_data.prices
            current_price = market_data.current_price

            # Generate multiple forecasts
            arima_forecast = self._arima_forecast(prices, current_price)
            prophet_forecast = self._prophet_forecast(prices, forecast_horizon)
            lstm_forecast = self._lstm_forecast(prices, current_price)

            # Ensemble forecast
            ensemble_forecast = np.mean([arima_forecast, prophet_forecast, lstm_forecast])

            # Calculate confidence
            forecast_std = np.std([arima_forecast, prophet_forecast, lstm_forecast])
            volatility = market_data.volatility_20d
            confidence = max(0.3, min(0.9, 1.0 - (forecast_std / current_price) - volatility * 0.5))

            # Prediction intervals
            lower_bound = ensemble_forecast - 1.96 * forecast_std
            upper_bound = ensemble_forecast + 1.96 * forecast_std

            # Generate simulated return sequence for risk analysis
            simulated_returns = self._generate_simulated_returns(prices, forecast_horizon)

            # Calculate probabilities
            expected_return = (ensemble_forecast - current_price) / current_price
            upside_probability = max(0.1, min(0.9, 0.5 + expected_return))
            downside_risk = 1.0 - upside_probability

            # Forecast volatility
            volatility_forecast = self._forecast_volatility(prices)

            # Calculate accuracy score
            trend_consistency = self._calculate_trend_consistency(prices)
            accuracy_score = confidence * trend_consistency

            forecast_data = ForecastData(
                arima_forecast=arima_forecast,
                prophet_forecast=prophet_forecast,
                lstm_forecast=lstm_forecast,
                ensemble_forecast=ensemble_forecast,
                forecast_confidence=confidence,
                prediction_interval=[lower_bound, upper_bound],
                forecast_horizon_days=forecast_horizon,
                forecast_accuracy_score=accuracy_score,
                upside_probability=upside_probability,
                downside_risk=downside_risk,
                volatility_forecast=volatility_forecast,
                simulated_returns=simulated_returns
            )

            state['forecast_data'] = forecast_data

            price_change = ((ensemble_forecast - current_price) / current_price) * 100
            print(f"✅ {self.name}: Ensemble forecast: ${ensemble_forecast:.2f} ({price_change:+.1f}%)")
            print(f"   Confidence: {confidence:.1%}")
            print(f"   Upside Probability: {upside_probability:.1%}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _arima_forecast(self, prices: pd.Series, current_price: float) -> float:
        """Simple ARIMA-like forecast using moving averages"""
        if len(prices) >= 20:
            ma_5 = prices.tail(5).mean()
            ma_20 = prices.tail(20).mean()
            trend_factor = (ma_5 - ma_20) / ma_20

            # Add momentum factor
            momentum = (prices.iloc[-1] - prices.iloc[-5]) / prices.iloc[-5]

            return current_price * (1 + trend_factor * 0.5 + momentum * 0.3)
        return current_price * 1.01

    def _prophet_forecast(self, prices: pd.Series, horizon: int) -> float:
        """Prophet forecast (simplified if Prophet unavailable)"""
        if HAS_PROPHET and len(prices) >= 30:
            try:
                df = pd.DataFrame({
                    'ds': prices.index,
                    'y': prices.values
                })

                model = Prophet(
                    daily_seasonality=True,
                    yearly_seasonality=False,
                    weekly_seasonality=True,
                    changepoint_prior_scale=0.05
                )
                model.fit(df)

                future = model.make_future_dataframe(periods=horizon)
                forecast = model.predict(future)

                return float(forecast['yhat'].iloc[-1])
            except:
                pass

        # Fallback method - seasonal adjustment
        if len(prices) >= 7:
            # Simple weekly seasonality
            day_of_week = len(prices) % 7
            seasonal_factor = 1.0 + np.random.normal(0, 0.01)  # Small seasonal adjustment
            return prices.iloc[-1] * seasonal_factor * (1 + np.random.normal(0.02, 0.05))

        return prices.iloc[-1] * (1 + np.random.normal(0.02, 0.05))

    def _lstm_forecast(self, prices: pd.Series, current_price: float) -> float:
        """LSTM-like forecast using exponential smoothing"""
        if len(prices) >= 20:
            # Use multiple lookback windows for LSTM simulation
            weights_short = np.exp(np.linspace(-2, 0, 5))
            weights_short = weights_short / weights_short.sum()

            weights_long = np.exp(np.linspace(-3, 0, 15))
            weights_long = weights_long / weights_long.sum()

            short_term = np.sum(prices.tail(5) * weights_short)
            long_term = np.sum(prices.tail(15) * weights_long)

            # Combine with volatility adjustment
            volatility = prices.pct_change().tail(20).std()
            noise = np.random.normal(0, volatility * 0.1)

            return (short_term * 0.7 + long_term * 0.3) * (1 + noise)

        return current_price * (1 + np.random.normal(0.02, 0.05))

    def _generate_simulated_returns(self, prices: pd.Series, horizon: int) -> List[float]:
        """Generate simulated return sequence for risk analysis"""
        returns = prices.pct_change().dropna()

        if len(returns) < 20:
            # Not enough data, use simple simulation
            mean_return = 0.001  # 0.1% daily
            std_return = 0.02    # 2% daily volatility
        else:
            mean_return = returns.mean()
            std_return = returns.std()

        # Generate random returns for forecast horizon (extended for Monte Carlo)
        simulated_returns = np.random.normal(mean_return, std_return, horizon * 50).tolist()

        return simulated_returns

    def _forecast_volatility(self, prices: pd.Series) -> float:
        """Forecast future volatility using GARCH-like approach"""
        returns = prices.pct_change().dropna()
        if len(returns) < 20:
            return 0.2

        # Simple volatility forecasting
        recent_vol = returns.tail(10).std() * np.sqrt(252)
        long_term_vol = returns.std() * np.sqrt(252)

        # Weighted average with more weight on recent
        forecast_vol = 0.7 * recent_vol + 0.3 * long_term_vol

        return min(1.0, max(0.05, forecast_vol))

    def _calculate_trend_consistency(self, prices: pd.Series) -> float:
        """Calculate trend consistency for confidence adjustment"""
        if len(prices) < 10:
            return 0.5

        # R-squared of linear regression for trend strength
        x = np.arange(len(prices.tail(20)))
        y = prices.tail(20).values

        try:
            correlation = np.corrcoef(x, y)[0, 1]
            r_squared = correlation ** 2
            return min(1.0, r_squared + 0.3)  # Add base confidence
        except:
            return 0.5

In [6]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 5: Agent 4 - Macro Economic Agent (FIXED - CAPSTONE REQUIREMENT #2)

# =============================================================================
# AGENT 4: MACRO ECONOMIC AGENT (FIXED)
# =============================================================================

class MacroEconomicAgent:
    """Macro-economic analysis agent - FIXED naming for StrategistAgent integration"""

    def __init__(self):
        self.name = "MacroEconomicAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict) -> Dict:
        """Analyze macro-economic factors and market regime"""

        try:
            print(f"🌍 {self.name}: Analyzing macro-economic factors...")

            # In production, these would come from FRED API, Bloomberg, etc.
            # For demo, we simulate realistic macro data

            macro_data = MacroData(
                gdp_growth=np.random.normal(2.5, 0.5),  # ~2.5% GDP growth
                inflation_rate=np.random.normal(3.2, 0.3),  # ~3.2% inflation
                unemployment_rate=np.random.normal(3.8, 0.2),  # ~3.8% unemployment
                federal_funds_rate=np.random.normal(5.25, 0.25),  # ~5.25% fed rate
                vix=np.random.normal(18, 5),  # VIX around 18
                dollar_index=np.random.normal(103, 2),  # DXY around 103
                market_sentiment=np.random.choice(["bullish", "neutral", "bearish"], p=[0.3, 0.4, 0.3]),

                # Enhanced macro indicators
                yield_curve_slope=np.random.normal(1.5, 0.3),  # 10Y-2Y spread
                credit_spreads=np.random.normal(1.2, 0.2),  # Investment grade spreads
                economic_surprise_index=np.random.normal(0.0, 0.5)  # Economic surprise index
            )

            # Store in state dictionary
            state['macro_data'] = macro_data

            print(f"✅ {self.name}: Analysis complete")
            print(f"   Market Sentiment: {macro_data.market_sentiment}")
            print(f"   GDP Growth: {macro_data.gdp_growth:.1f}%")
            print(f"   Inflation: {macro_data.inflation_rate:.1f}%")
            print(f"   Fed Funds Rate: {macro_data.federal_funds_rate:.2f}%")
            print(f"   VIX: {macro_data.vix:.1f}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _analyze_market_regime(self, vix: float, yield_curve: float, credit_spreads: float) -> str:
        """Analyze current market regime based on key indicators"""

        # Simple regime classification
        if vix > 25 and credit_spreads > 2.0:
            return "risk_off"
        elif vix < 15 and yield_curve > 1.0:
            return "risk_on"
        elif yield_curve < 0:
            return "recession_warning"
        else:
            return "neutral"

    def _calculate_recession_probability(self, unemployment_rate: float,
                                       yield_curve: float, credit_spreads: float) -> float:
        """Simple recession probability model"""

        # Sahm Rule component (unemployment rate rising)
        unemployment_score = max(0, unemployment_rate - 3.5) * 0.1

        # Yield curve inversion
        yield_curve_score = max(0, -yield_curve) * 0.3

        # Credit stress
        credit_score = max(0, credit_spreads - 1.5) * 0.2

        # Combine scores
        recession_prob = min(1.0, unemployment_score + yield_curve_score + credit_score)

        return recession_prob

In [7]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 6: Agent 5 - Sentiment Agent

# =============================================================================
# AGENT 5: SENTIMENT AGENT
# =============================================================================

class SentimentAgent:
    """Market sentiment analysis agent"""

    def __init__(self):
        self.name = "SentimentAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict) -> Dict:
        """Analyze market sentiment from multiple sources"""

        try:
            print(f"💭 {self.name}: Analyzing market sentiment...")

            symbol = state.get('symbol', 'UNKNOWN')

            # In production, these would come from:
            # - News API (NewsAPI, Alpha Vantage, etc.)
            # - Social media APIs (Twitter, Reddit, etc.)
            # - Financial news sentiment (Bloomberg, Reuters)
            # - Analyst ratings aggregation

            # Simulate sentiment analysis with realistic distributions
            news_sentiment = np.random.normal(0.1, 0.3)  # Slightly positive bias in news
            social_sentiment = np.random.normal(0.0, 0.4)  # More volatile social sentiment

            # Overall sentiment is weighted average
            overall_sentiment = (news_sentiment * 0.6 + social_sentiment * 0.4)

            # Determine sentiment trend
            if overall_sentiment > 0.2:
                sentiment_trend = "positive"
            elif overall_sentiment < -0.2:
                sentiment_trend = "negative"
            else:
                sentiment_trend = "neutral"

            # Simulate key topics (in production, use NLP topic modeling)
            all_topics = [
                "earnings", "product_launch", "market_conditions",
                "regulation", "competition", "economic_data",
                "fed_policy", "geopolitical", "sector_rotation"
            ]
            key_topics = np.random.choice(all_topics, size=np.random.randint(2, 5), replace=False).tolist()

            # Enhanced sentiment metrics
            sentiment_momentum = np.random.normal(0.0, 0.2)  # Change in sentiment
            fear_greed_index = np.random.uniform(20, 80)  # CNN Fear & Greed style index
            analyst_rating_trend = np.random.choice(["upgrade", "downgrade", "neutral"], p=[0.3, 0.2, 0.5])

            # Confidence score based on data sources and consistency
            confidence_score = np.random.uniform(0.6, 0.9)

            sentiment_data = SentimentData(
                news_sentiment=news_sentiment,
                social_media_sentiment=social_sentiment,
                overall_sentiment=overall_sentiment,
                sentiment_trend=sentiment_trend,
                confidence_score=confidence_score,
                key_topics=key_topics,
                sentiment_momentum=sentiment_momentum,
                fear_greed_index=fear_greed_index,
                analyst_rating_trend=analyst_rating_trend
            )

            # Store in state dictionary
            state['sentiment_data'] = sentiment_data

            print(f"✅ {self.name}: Analysis complete")
            print(f"   Overall Sentiment: {sentiment_trend} ({overall_sentiment:.2f})")
            print(f"   Fear & Greed Index: {fear_greed_index:.0f}")
            print(f"   Key Topics: {', '.join(key_topics[:3])}")
            print(f"   Analyst Trend: {analyst_rating_trend}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _analyze_news_sentiment(self, symbol: str) -> float:
        """Analyze news sentiment for specific symbol"""
        # In production, this would:
        # 1. Fetch recent news articles for the symbol
        # 2. Use NLP sentiment analysis (VADER, TextBlob, or transformer models)
        # 3. Aggregate sentiment scores across articles
        # 4. Weight by article source credibility and recency

        # For demo, simulate realistic news sentiment
        return np.random.normal(0.1, 0.3)

    def _analyze_social_sentiment(self, symbol: str) -> float:
        """Analyze social media sentiment for specific symbol"""
        # In production, this would:
        # 1. Scrape Twitter, Reddit, StockTwits mentions
        # 2. Filter for relevant content and remove spam
        # 3. Apply sentiment analysis to social posts
        # 4. Weight by user influence and engagement

        # For demo, simulate more volatile social sentiment
        return np.random.normal(0.0, 0.4)

    def _calculate_fear_greed_index(self, vix: float, market_momentum: float,
                                  put_call_ratio: float = None) -> float:
        """Calculate Fear & Greed Index similar to CNN's"""

        # VIX component (inverse relationship)
        vix_score = max(0, min(100, (35 - vix) * 100 / 25))

        # Market momentum component
        momentum_score = max(0, min(100, (market_momentum + 0.1) * 500))

        # Put/Call ratio component (if available)
        if put_call_ratio:
            pc_score = max(0, min(100, (1.5 - put_call_ratio) * 100))
        else:
            pc_score = 50  # Neutral if not available

        # Simple average (in production, use weighted average)
        fear_greed = (vix_score + momentum_score + pc_score) / 3

        return fear_greed

In [22]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 7: Agent 6 - Fixed Strategist Agent (CAPSTONE REQUIREMENT #2)

# =============================================================================
# AGENT 6: STRATEGIST AGENT (FIXED - GPT-4 ENHANCED)
# =============================================================================

# =============================================================================
# STRATEGIST AGENT FIX - ADD MISSING METHODS
# =============================================================================

# Add these methods to your existing StrategistAgent class:

class StrategistAgent:
    """AI-powered trading strategist with GPT-4 integration - COMPLETE VERSION"""

    def __init__(self, api_key: Optional[str] = None):
        self.name = "StrategistAgent"
        self.client = None
        self.has_openai = False

        if api_key and HAS_OPENAI:
            try:
                self.client = OpenAI(api_key=api_key)
                self.has_openai = True
                print(f"✅ {self.name} initialized with GPT-4")
            except Exception as e:
                print(f"⚠️ GPT initialization failed: {e}")
                self.has_openai = False
        else:
            print(f"✅ {self.name} initialized with rule-based logic")

    async def process(self, state: Dict) -> Dict:
        """Generate personalized recommendation using all agent data"""

        try:
            print(f"🧠 {self.name}: Generating AI recommendation...")

            # Verify we have the required data from other agents
            required_data = ['market_data']
            missing_data = [key for key in required_data if key not in state or not state[key]]

            if missing_data:
                print(f"⚠️ {self.name}: Missing required data: {missing_data}")
                return state

            # Generate recommendation using GPT-4 or fallback logic
            if self.has_openai and self.client:
                try:
                    recommendation = await self._generate_gpt_recommendation(state)
                    print(f"✅ {self.name}: GPT-4 recommendation generated")
                except Exception as e:
                    print(f"⚠️ GPT-4 failed, using enhanced fallback: {e}")
                    recommendation = self._generate_enhanced_recommendation(state)
            else:
                recommendation = self._generate_enhanced_recommendation(state)

            # Store recommendation in state
            state['recommendation'] = recommendation

            print(f"✅ {self.name}: {recommendation.action} recommendation")
            print(f"   Confidence: {recommendation.confidence:.1%}")
            print(f"   Position Size: {recommendation.position_size:.1%}")
            print(f"   Risk Level: {recommendation.risk_level}")
            print(f"   Risk/Reward: {recommendation.risk_reward_ratio:.2f}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _generate_enhanced_recommendation(self, state: Dict) -> PersonalizedRecommendation:
        """Generate recommendation using enhanced rule-based logic with all agent data"""

        # Get current price
        current_price = 100.0  # Default
        if 'market_data' in state and state['market_data']:
            current_price = state['market_data'].current_price

        # Calculate comprehensive scores using all available agent data
        technical_score = self._calculate_technical_score(state)
        forecast_score = self._calculate_forecast_score(state)
        risk_score = self._calculate_risk_score(state)
        macro_score = self._calculate_macro_score(state)
        sentiment_score = self._calculate_sentiment_score(state)

        # Weighted overall score
        overall_score = (
            technical_score * 0.25 +
            forecast_score * 0.25 +
            risk_score * 0.20 +
            macro_score * 0.15 +
            sentiment_score * 0.15
        )

        # Enhanced decision logic
        if overall_score > 0.6:
            action = "BUY"
            confidence = min(0.95, 0.6 + overall_score * 0.3)
            position_size = min(0.4, confidence * 0.5)
            risk_level = "LOW" if confidence > 0.8 else "MEDIUM"
            time_horizon = "MEDIUM"
        elif overall_score < -0.6:
            action = "SELL"
            confidence = min(0.95, 0.6 + abs(overall_score) * 0.3)
            position_size = min(0.3, confidence * 0.4)
            risk_level = "MEDIUM" if confidence > 0.7 else "HIGH"
            time_horizon = "SHORT"
        else:
            action = "HOLD"
            confidence = 0.6 + abs(overall_score) * 0.2
            position_size = 0.0
            risk_level = "LOW"
            time_horizon = "LONG"

        # Generate comprehensive analysis components with error handling
        detailed_reasoning = self._generate_detailed_reasoning(state, overall_score)

        try:
            risk_factors, opportunity_factors = self._identify_risk_opportunity_factors(state)
        except:
            risk_factors = ["High volatility environment", "Technical overbought conditions"]
            opportunity_factors = ["Strong trend momentum", "Forecast confidence"]

        try:
            alternative_scenarios = self._generate_alternative_scenarios(state)
        except:
            alternative_scenarios = {
                "bull_case": "Technical momentum continues with improving fundamentals",
                "bear_case": "Risk factors materialize leading to correction",
                "base_case": "Mixed signals result in sideways price action"
            }

        try:
            portfolio_impact = self._analyze_portfolio_impact(state, action, position_size)
        except:
            portfolio_impact = f"{action} position of {position_size:.1%} would impact portfolio risk profile"

        try:
            market_timing_analysis = self._analyze_market_timing(state)
        except:
            market_timing_analysis = "Mixed timing signals suggest cautious approach"

        # Calculate risk metrics
        risk_reward_ratio = self._calculate_risk_reward_ratio(current_price, action, state)
        probability_of_success = min(0.9, confidence + 0.1)
        max_drawdown_estimate = self._estimate_maximum_drawdown(state)

        return PersonalizedRecommendation(
            action=action,
            confidence=confidence,
            position_size=position_size,
            entry_price=current_price,
            stop_loss=current_price * (0.92 if action == "BUY" else 1.08),
            take_profit=current_price * (1.25 if action == "BUY" else 0.75),
            risk_level=risk_level,
            time_horizon=time_horizon,
            detailed_reasoning=detailed_reasoning,
            key_risk_factors=risk_factors,
            key_opportunity_factors=opportunity_factors,
            alternative_scenarios=alternative_scenarios,
            portfolio_impact=portfolio_impact,
            market_timing_analysis=market_timing_analysis,
            risk_reward_ratio=risk_reward_ratio,
            probability_of_success=probability_of_success,
            maximum_drawdown_estimate=max_drawdown_estimate
        )

    # Add all the missing helper methods:

    def _calculate_technical_score(self, state: Dict) -> float:
        """Calculate technical analysis score from market data"""
        if 'market_data' not in state or not state['market_data']:
            return 0.0

        md = state['market_data']
        score = 0.0

        # Trend analysis (40% of technical score)
        trend_scores = {
            "strongly_bullish": 1.0, "bullish": 0.5, "neutral": 0.0,
            "bearish": -0.5, "strongly_bearish": -1.0
        }
        score += trend_scores.get(md.trend, 0.0) * 0.4

        # RSI analysis (30% of technical score)
        if md.rsi < 30:
            score += 0.3  # Oversold - bullish
        elif md.rsi > 70:
            score -= 0.3  # Overbought - bearish
        elif 40 <= md.rsi <= 60:
            score += 0.1  # Neutral zone - slightly positive

        # MACD signal (20% of technical score)
        macd_scores = {"bullish": 0.2, "bearish": -0.2, "neutral": 0.0}
        score += macd_scores.get(md.macd_signal, 0.0)

        # Volume trend (10% of technical score)
        volume_scores = {"increasing": 0.1, "stable": 0.0, "decreasing": -0.05}
        score += volume_scores.get(md.volume_trend, 0.0)

        return max(-1.0, min(1.0, score))

    def _calculate_forecast_score(self, state: Dict) -> float:
        """Calculate score based on forecasting data"""
        if 'forecast_data' not in state or not state['forecast_data']:
            return 0.0

        fd = state['forecast_data']
        current_price = state.get('market_data', {}).current_price if 'market_data' in state else 100

        # Expected price change
        price_change = (fd.ensemble_forecast - current_price) / current_price
        confidence_weighted_change = price_change * fd.forecast_confidence
        upside_bonus = (fd.upside_probability - 0.5) * 0.5
        volatility_penalty = -fd.volatility_forecast * 0.2

        score = confidence_weighted_change + upside_bonus + volatility_penalty
        return max(-1.0, min(1.0, score * 2))

    def _calculate_risk_score(self, state: Dict) -> float:
        """Calculate risk-adjusted score from risk metrics"""
        if 'risk_metrics' not in state or not state['risk_metrics']:
            return 0.0

        rm = state['risk_metrics']
        score = 0.0

        # Sharpe ratio component
        if rm.sharpe_ratio > 1.5:
            score += 0.5
        elif rm.sharpe_ratio > 1.0:
            score += 0.25
        elif rm.sharpe_ratio < 0:
            score -= 0.5

        # Volatility component
        if rm.portfolio_volatility > 0.4:
            score -= 0.3
        elif rm.portfolio_volatility < 0.15:
            score += 0.15

        # Drawdown component
        if rm.maximum_drawdown < -0.3:
            score -= 0.2
        elif rm.maximum_drawdown > -0.1:
            score += 0.1

        return max(-1.0, min(1.0, score))

    def _calculate_macro_score(self, state: Dict) -> float:
        """Calculate macro-economic score"""
        if 'macro_data' not in state or not state['macro_data']:
            return 0.0

        md = state['macro_data']
        score = 0.0

        # Market sentiment
        sentiment_scores = {"bullish": 0.5, "neutral": 0.0, "bearish": -0.5}
        score += sentiment_scores.get(md.market_sentiment, 0.0)

        # VIX analysis
        if md.vix < 15:
            score += 0.3
        elif md.vix > 30:
            score -= 0.3

        # Economic indicators
        if md.gdp_growth > 3.0:
            score += 0.1
        elif md.gdp_growth < 1.0:
            score -= 0.1

        return max(-1.0, min(1.0, score))

    def _calculate_sentiment_score(self, state: Dict) -> float:
        """Calculate sentiment-based score"""
        if 'sentiment_data' not in state or not state['sentiment_data']:
            return 0.0

        sd = state['sentiment_data']
        sentiment_score = sd.overall_sentiment * sd.confidence_score
        fear_greed_factor = (sd.fear_greed_index - 50) / 100
        analyst_scores = {"upgrade": 0.2, "neutral": 0.0, "downgrade": -0.2}
        analyst_score = analyst_scores.get(sd.analyst_rating_trend, 0.0)

        total_score = sentiment_score * 0.6 + fear_greed_factor * 0.3 + analyst_score * 0.1
        return max(-1.0, min(1.0, total_score))

    def _generate_detailed_reasoning(self, state: Dict, overall_score: float) -> str:
        """Generate comprehensive reasoning using all agent data"""
        reasoning = f"Multi-agent analysis yields overall score of {overall_score:.2f}.\n\n"

        if 'market_data' in state and state['market_data']:
            md = state['market_data']
            reasoning += f"TECHNICAL: {md.trend} trend, RSI {md.rsi:.1f}, {md.macd_signal} MACD.\n"

        if 'risk_metrics' in state and state['risk_metrics']:
            rm = state['risk_metrics']
            reasoning += f"RISK: Sharpe {rm.sharpe_ratio:.2f}, volatility {rm.portfolio_volatility:.1%}.\n"

        if 'forecast_data' in state and state['forecast_data']:
            fd = state['forecast_data']
            reasoning += f"FORECAST: ${fd.ensemble_forecast:.2f} target, {fd.forecast_confidence:.1%} confidence.\n"

        return reasoning

    def _identify_risk_opportunity_factors(self, state: Dict) -> tuple:
        """Identify key risk and opportunity factors"""
        risk_factors = []
        opportunity_factors = []

        if 'market_data' in state and state['market_data']:
            md = state['market_data']
            if md.rsi > 70:
                risk_factors.append("Overbought conditions (RSI > 70)")
            elif md.rsi < 30:
                opportunity_factors.append("Oversold conditions")

        if 'risk_metrics' in state and state['risk_metrics']:
            rm = state['risk_metrics']
            if rm.portfolio_volatility > 0.3:
                risk_factors.append("High volatility")
            if rm.sharpe_ratio > 1.0:
                opportunity_factors.append("Strong risk-adjusted returns")

        return risk_factors[:3], opportunity_factors[:3]

    def _generate_alternative_scenarios(self, state: Dict) -> dict:
        """Generate bull, bear, and base case scenarios"""
        return {
            "bull_case": "Technical momentum and positive catalysts drive upside",
            "bear_case": "Risk factors materialize leading to correction",
            "base_case": "Mixed signals result in sideways action"
        }

    def _analyze_portfolio_impact(self, state: Dict, action: str, position_size: float) -> str:
        """Analyze impact on overall portfolio"""
        return f"{action} position of {position_size:.1%} would impact portfolio risk and diversification."

    def _analyze_market_timing(self, state: Dict) -> str:
        """Analyze market timing considerations"""
        timing_factors = []

        if 'market_data' in state and state['market_data']:
            md = state['market_data']
            if md.rsi > 70:
                timing_factors.append("overbought conditions suggest caution")
            elif md.rsi < 30:
                timing_factors.append("oversold conditions favor entry")

        if timing_factors:
            return "Timing analysis: " + "; ".join(timing_factors)
        else:
            return "Mixed timing signals suggest neutral approach"

    def _calculate_risk_reward_ratio(self, current_price: float, action: str, state: Dict) -> float:
        """Calculate risk/reward ratio for the recommendation"""
        if action == "HOLD":
            return 1.0

        # Default 2:1 reward/risk ratio
        base_ratio = 2.0

        # Adjust based on volatility
        if 'risk_metrics' in state and state['risk_metrics']:
            volatility = state['risk_metrics'].portfolio_volatility
            if volatility > 0.3:  # High volatility
                base_ratio = 1.5
            elif volatility < 0.15:  # Low volatility
                base_ratio = 2.5

        return base_ratio

    def _estimate_maximum_drawdown(self, state: Dict) -> float:
        """Estimate maximum drawdown for the position"""
        if 'risk_metrics' in state and state['risk_metrics']:
            return abs(state['risk_metrics'].maximum_drawdown)
        return 0.15  # Default 15%

# =============================================================================
# HOW TO USE THE FIX
# =============================================================================

# Option 1: Replace your existing StrategistAgent class with the one above

# Option 2: If you want to patch your existing class, add this to your code:
"""
# Add missing methods to existing StrategistAgent
def patch_strategist_agent():
    # Copy all the methods from above and add them to your existing StrategistAgent class
    pass
"""

print("✅ StrategistAgent fix ready!")
print("Replace your existing StrategistAgent class with the complete version above.")

✅ StrategistAgent fix ready!
Replace your existing StrategistAgent class with the complete version above.


In [11]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 8: Agent 7 - Financial Planner Agent

# =============================================================================
# AGENT 7: FINANCIAL PLANNER AGENT
# =============================================================================

class FinancialPlannerAgent:
    """Financial planning agent with Sharpe ratio and risk-adjusted planning"""

    def __init__(self):
        self.name = "FinancialPlannerAgent"

        # Asset allocation strategies
        self.strategies = {
            "conservative": {
                "us_stocks": 0.30, "international_stocks": 0.10,
                "bonds": 0.55, "cash": 0.05
            },
            "moderate": {
                "us_stocks": 0.50, "international_stocks": 0.20,
                "bonds": 0.25, "cash": 0.05
            },
            "aggressive": {
                "us_stocks": 0.60, "international_stocks": 0.25,
                "bonds": 0.10, "cash": 0.05
            }
        }

        # Expected returns and volatilities (annual)
        self.asset_assumptions = {
            "us_stocks": {"return": 0.10, "volatility": 0.16},
            "international_stocks": {"return": 0.08, "volatility": 0.18},
            "bonds": {"return": 0.04, "volatility": 0.06},
            "cash": {"return": 0.02, "volatility": 0.01}
        }

        print(f"✅ {self.name} initialized")

    async def process(self, state: Dict, goal: FinancialGoal) -> Dict:
        """Create comprehensive financial plan with risk-adjusted metrics"""

        try:
            print(f"💰 {self.name}: Creating financial plan...")

            # Calculate projections
            projections = self._calculate_projections(goal)

            # Optimize asset allocation
            asset_allocation = self._optimize_allocation(goal)

            # Calculate plan-level risk metrics (CAPSTONE REQUIREMENT #4)
            plan_sharpe = self._calculate_plan_sharpe(asset_allocation)
            plan_volatility = self._calculate_plan_volatility(asset_allocation)
            plan_drawdown = self._estimate_plan_drawdown(asset_allocation)

            # Tax optimization
            tax_optimization = self._optimize_taxes(goal)

            # Monthly breakdown
            monthly_breakdown = self._calculate_monthly_breakdown(goal, asset_allocation)

            # Monte Carlo simulation with risk metrics
            monte_carlo = self._run_monte_carlo(goal, asset_allocation, 1000)

            # Generate actionable recommendations
            recommendations = self._generate_recommendations(goal, projections, monte_carlo)

            # Create comprehensive financial plan result
            plan_result = FinancialPlanResult(
                goal=goal,
                projected_value=projections['projected_value'],
                success_probability=projections['success_probability'],
                required_monthly=projections['required_monthly'],
                asset_allocation=asset_allocation,
                tax_optimization=tax_optimization,
                monthly_breakdown=monthly_breakdown,
                recommendations=recommendations,
                is_achievable=projections['is_achievable'],
                monte_carlo_results=monte_carlo,
                plan_sharpe_ratio=plan_sharpe,
                plan_max_drawdown=plan_drawdown,
                plan_volatility=plan_volatility
            )

            # Store in state dictionary
            state['financial_plan'] = plan_result

            print(f"✅ {self.name}: Financial plan created")
            print(f"   Success Probability: {projections['success_probability']:.1%}")
            print(f"   Plan Sharpe Ratio: {plan_sharpe:.2f}")
            print(f"   Expected Volatility: {plan_volatility:.1%}")
            print(f"   Est. Max Drawdown: {plan_drawdown:.1%}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")

        return state

    def _calculate_projections(self, goal: FinancialGoal) -> Dict[str, float]:
        """Calculate financial projections with compound growth"""

        # Get portfolio expected return based on risk tolerance
        allocation = self.strategies[goal.risk_tolerance]
        portfolio_return = sum(
            allocation[asset] * self.asset_assumptions[asset]["return"]
            for asset in allocation
        )

        # Future value calculations
        years = goal.time_horizon_years
        monthly_rate = portfolio_return / 12
        months = years * 12

        # Future value of current amount (compound growth)
        fv_current = goal.current_amount * (1 + portfolio_return) ** years

        # Future value of monthly contributions (annuity formula)
        if monthly_rate > 0:
            fv_contributions = goal.monthly_contribution * (
                ((1 + monthly_rate) ** months - 1) / monthly_rate
            )
        else:
            fv_contributions = goal.monthly_contribution * months

        # Total projected value
        projected_value = fv_current + fv_contributions

        # Success analysis
        success_probability = min(1.0, projected_value / goal.target_amount)
        gap = goal.target_amount - projected_value

        # Required additional monthly contribution if not achievable
        if gap > 0 and monthly_rate > 0:
            required_monthly = gap / (((1 + monthly_rate) ** months - 1) / monthly_rate)
        else:
            required_monthly = 0

        return {
            'projected_value': projected_value,
            'success_probability': success_probability,
            'required_monthly': required_monthly,
            'is_achievable': gap <= 0,
            'portfolio_return': portfolio_return
        }

    def _optimize_allocation(self, goal: FinancialGoal) -> Dict[str, float]:
        """Optimize asset allocation based on goal parameters"""

        base_allocation = self.strategies[goal.risk_tolerance].copy()

        # Age-based adjustments (Rule of 100 - Age)
        target_equity_ratio = max(0.3, min(0.9, (100 - goal.age) / 100))
        current_equity_ratio = base_allocation["us_stocks"] + base_allocation["international_stocks"]

        equity_adjustment = target_equity_ratio - current_equity_ratio

        # Adjust if significant difference
        if abs(equity_adjustment) > 0.05:
            if equity_adjustment > 0:  # Need more equity
                base_allocation["us_stocks"] += equity_adjustment * 0.7
                base_allocation["international_stocks"] += equity_adjustment * 0.3
                base_allocation["bonds"] -= equity_adjustment * 0.8
                base_allocation["cash"] -= equity_adjustment * 0.2
            else:  # Need less equity
                reduction = abs(equity_adjustment)
                base_allocation["us_stocks"] -= reduction * 0.7
                base_allocation["international_stocks"] -= reduction * 0.3
                base_allocation["bonds"] += reduction * 0.8
                base_allocation["cash"] += reduction * 0.2

        # Time horizon adjustments
        if goal.time_horizon_years > 20:
            # Very long term - increase growth
            base_allocation["us_stocks"] += 0.05
            base_allocation["bonds"] -= 0.05
        elif goal.time_horizon_years < 5:
            # Short term - increase safety
            base_allocation["us_stocks"] -= 0.10
            base_allocation["bonds"] += 0.07
            base_allocation["cash"] += 0.03

        # Goal type adjustments
        if goal.goal_type == "house":
            # House down payment - more conservative
            base_allocation["cash"] += 0.10
            base_allocation["bonds"] += 0.05
            base_allocation["us_stocks"] -= 0.15
        elif goal.goal_type == "education":
            # Education funding - target date approach
            if goal.time_horizon_years < 10:
                base_allocation["bonds"] += 0.10
                base_allocation["us_stocks"] -= 0.10

        # Ensure all allocations are non-negative and normalize
        for key in base_allocation:
            base_allocation[key] = max(0, base_allocation[key])

        # Normalize to sum to 1.0
        total = sum(base_allocation.values())
        if total > 0:
            base_allocation = {k: v/total for k, v in base_allocation.items()}

        return base_allocation

    def _calculate_plan_sharpe(self, allocation: Dict[str, float]) -> float:
        """Calculate expected Sharpe ratio for the financial plan"""

        # Portfolio expected return
        portfolio_return = sum(
            allocation[asset] * self.asset_assumptions[asset]["return"]
            for asset in allocation
        )

        # Portfolio volatility (simplified - assumes no correlation)
        portfolio_variance = sum(
            (allocation[asset] ** 2) * (self.asset_assumptions[asset]["volatility"] ** 2)
            for asset in allocation
        )
        portfolio_volatility = np.sqrt(portfolio_variance)

        # Sharpe ratio calculation (risk-free rate = 2%)
        risk_free_rate = 0.02
        if portfolio_volatility > 0:
            sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        else:
            sharpe_ratio = 0.0

        return sharpe_ratio

    def _calculate_plan_volatility(self, allocation: Dict[str, float]) -> float:
        """Calculate expected portfolio volatility"""

        portfolio_variance = sum(
            (allocation[asset] ** 2) * (self.asset_assumptions[asset]["volatility"] ** 2)
            for asset in allocation
        )

        return np.sqrt(portfolio_variance)

    def _estimate_plan_drawdown(self, allocation: Dict[str, float]) -> float:
        """Estimate maximum drawdown for the financial plan"""

        # Simple estimation based on portfolio volatility
        portfolio_volatility = self._calculate_plan_volatility(allocation)

        # Rule of thumb: max drawdown ≈ 2.5 * annual volatility
        estimated_drawdown = portfolio_volatility * 2.5

        # Cap between reasonable bounds
        return min(0.6, max(0.05, estimated_drawdown))

    def _optimize_taxes(self, goal: FinancialGoal) -> Dict[str, float]:
        """Optimize tax-advantaged account allocation"""

        annual_contribution = goal.monthly_contribution * 12

        # 2024 contribution limits with catch-up provisions
        max_401k = 23000 + (7500 if goal.age >= 50 else 0)
        max_ira = 7000 + (1000 if goal.age >= 50 else 0)

        # Income-based phase-outs (simplified)
        if goal.annual_income > 120000:
            max_ira = max(0, max_ira - (goal.annual_income - 120000) * 0.1)

        # Optimization strategy: maximize tax advantages
        remaining_contribution = annual_contribution

        # 1. Maximize 401(k) first (up to employer match, then fill other accounts)
        optimal_401k = min(max_401k, remaining_contribution * 0.6)  # Assume up to 60% to 401k
        remaining_contribution -= optimal_401k

        # 2. Fill IRA next
        optimal_ira = min(max_ira, remaining_contribution)
        remaining_contribution -= optimal_ira

        # 3. Additional 401(k) if room remains
        additional_401k = min(max_401k - optimal_401k, remaining_contribution)
        optimal_401k += additional_401k
        remaining_contribution -= additional_401k

        # 4. Remainder goes to taxable account
        optimal_taxable = remaining_contribution

        # Calculate tax savings
        tax_deductible = optimal_401k + optimal_ira
        tax_savings = tax_deductible * goal.tax_rate

        return {
            "401k_annual": optimal_401k,
            "401k_monthly": optimal_401k / 12,
            "ira_annual": optimal_ira,
            "ira_monthly": optimal_ira / 12,
            "taxable_annual": optimal_taxable,
            "taxable_monthly": optimal_taxable / 12,
            "tax_savings": tax_savings,
            "marginal_rate": goal.tax_rate
        }

    def _calculate_monthly_breakdown(self, goal: FinancialGoal,
                                   allocation: Dict[str, float]) -> Dict[str, float]:
        """Calculate monthly contribution breakdown by asset class"""

        monthly = goal.monthly_contribution

        breakdown = {
            "total_monthly": monthly
        }

        # Add monthly allocation for each asset class
        for asset, percentage in allocation.items():
            breakdown[f"{asset}_monthly"] = monthly * percentage

        return breakdown

    def _run_monte_carlo(self, goal: FinancialGoal, allocation: Dict[str, float],
                        n_simulations: int = 1000) -> Dict[str, float]:
        """Run Monte Carlo simulation with enhanced risk metrics"""

        # Calculate portfolio characteristics
        portfolio_return = sum(
            allocation[asset] * self.asset_assumptions[asset]["return"]
            for asset in allocation
        )
        portfolio_volatility = self._calculate_plan_volatility(allocation)

        results = []
        drawdowns = []
        sharpe_ratios = []

        for _ in range(n_simulations):
            value = goal.current_amount
            peak_value = value
            max_drawdown = 0
            annual_returns = []

            for year in range(goal.time_horizon_years):
                # Add annual contributions
                value += goal.monthly_contribution * 12

                # Apply random annual return (normal distribution)
                annual_return = np.random.normal(portfolio_return, portfolio_volatility)
                annual_returns.append(annual_return)

                # Apply market stress scenarios (10% chance per year)
                if np.random.random() < 0.1:
                    stress_factor = np.random.uniform(0.7, 0.9)  # 10-30% stress
                    annual_return *= stress_factor

                value *= (1 + annual_return)

                # Track drawdown
                if value > peak_value:
                    peak_value = value
                else:
                    current_drawdown = (peak_value - value) / peak_value
                    max_drawdown = max(max_drawdown, current_drawdown)

                # Ensure value doesn't go negative
                value = max(0, value)

            # Calculate Sharpe ratio for this simulation
            if len(annual_returns) > 0:
                avg_return = np.mean(annual_returns)
                return_std = np.std(annual_returns)
                sim_sharpe = (avg_return - 0.02) / return_std if return_std > 0 else 0
                sharpe_ratios.append(sim_sharpe)

            results.append(value)
            drawdowns.append(max_drawdown)

        results = np.array(results)
        drawdowns = np.array(drawdowns)
        sharpe_ratios = np.array(sharpe_ratios)

        return {
            "mean": np.mean(results),
            "median": np.percentile(results, 50),
            "percentile_5": np.percentile(results, 5),
            "percentile_10": np.percentile(results, 10),
            "percentile_25": np.percentile(results, 25),
            "percentile_75": np.percentile(results, 75),
            "percentile_90": np.percentile(results, 90),
            "percentile_95": np.percentile(results, 95),
            "success_rate": np.mean(results >= goal.target_amount),
            "average_drawdown": np.mean(drawdowns),
            "worst_drawdown": np.max(drawdowns),
            "average_sharpe": np.mean(sharpe_ratios),
            "sharpe_std": np.std(sharpe_ratios)
        }

    def _generate_recommendations(self, goal: FinancialGoal, projections: Dict,
                                monte_carlo: Dict) -> List[str]:
        """Generate actionable financial planning recommendations"""

        recommendations = []

        # Goal achievability assessment
        if projections['is_achievable']:
            recommendations.append(f"✅ Goal is achievable! Projected: ${projections['projected_value']:,.0f}")
        else:
            shortfall = projections['required_monthly']
            recommendations.append(f"⚠️ Increase monthly contribution by ${shortfall:,.0f} to reach goal")

        # Monte Carlo insights
        success_rate = monte_carlo['success_rate']
        if success_rate > 0.8:
            recommendations.append("🎯 High probability of success based on Monte Carlo analysis")
        elif success_rate < 0.5:
            recommendations.append("📊 Consider extending timeline or increasing contributions")

        # Risk-adjusted recommendations
        avg_sharpe = monte_carlo.get('average_sharpe', 0)
        if avg_sharpe > 1.0:
            recommendations.append(f"📈 Excellent risk-adjusted returns expected (Sharpe: {avg_sharpe:.2f})")
        elif avg_sharpe < 0.5:
            recommendations.append("⚖️ Consider adjusting allocation for better risk-adjusted returns")

        # Age and time-based recommendations
        if goal.age < 35:
            recommendations.append("⏰ Young investor advantage - time is your greatest asset")
        elif goal.age > 45:
            recommendations.append("🎯 Focus on risk management while maintaining growth")

        # Time horizon recommendations
        if goal.time_horizon_years > 15:
            recommendations.append("📈 Long timeline allows for growth-oriented strategy")
        elif goal.time_horizon_years < 5:
            recommendations.append("🛡️ Short timeline requires conservative approach")

        # Risk tolerance recommendations
        if goal.risk_tolerance == "conservative" and goal.time_horizon_years > 10:
            recommendations.append("💡 Consider moderate risk for better long-term growth potential")

        # Practical action items
        recommendations.append("🔄 Rebalance portfolio quarterly to maintain target allocation")
        recommendations.append("📊 Review and adjust plan annually or after major life changes")

        return recommendations[:8]  # Limit to top 8 most relevant recommendations

In [17]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 9: Pipeline Orchestration and Main Execution

# =============================================================================
# PIPELINE ORCHESTRATION
# =============================================================================

async def run_pipeline(symbol="AAPL", openai_api_key=None, financial_goal=None):
    """
    Run the complete AI Financial Forecasting pipeline with all agents

    Args:
        symbol: Stock symbol to analyze (default: "AAPL")
        openai_api_key: OpenAI API key for enhanced recommendations (optional)
        financial_goal: FinancialGoal object for planning (optional)

    Returns:
        dict: Complete state with all agent results
    """

    print("🚀 STARTING AI FINANCIAL FORECASTING PIPELINE")
    print("=" * 60)

    # Initialize empty state
    state = {}

    # Initialize all agents
    print("\n📋 INITIALIZING AGENTS...")
    market_agent = MarketDataAgent()
    risk_agent = RiskAgent()
    forecast_agent = ForecastingAgent()
    macro_agent = MacroEconomicAgent()
    sentiment_agent = SentimentAgent()
    strategist_agent = StrategistAgent(api_key=openai_api_key)
    planner_agent = FinancialPlannerAgent()

    print(f"✅ All agents initialized successfully\n")

    try:
        # =================================================================
        # PHASE 1: DATA COLLECTION AND ANALYSIS
        # =================================================================
        print("📊 PHASE 1: DATA COLLECTION AND ANALYSIS")
        print("-" * 50)

        # Agent 1: Market Data (Foundation - required by others)
        print("\n🔄 Running Market Data Agent...")
        state = await market_agent.process(state, symbol=symbol, period="1y")

        # Agent 2: Risk Analysis (Uses market data)
        print("\n🔄 Running Risk Agent...")
        state = await risk_agent.process(state)

        # Agent 3: Forecasting (Uses market data)
        print("\n🔄 Running Forecasting Agent...")
        state = await forecast_agent.process(state, forecast_horizon=5)

        # =================================================================
        # PHASE 2: MACRO AND SENTIMENT ANALYSIS
        # =================================================================
        print("\n🌍 PHASE 2: MACRO AND SENTIMENT ANALYSIS")
        print("-" * 50)

        # Agent 4: Macro Economic Analysis
        print("\n🔄 Running Macro Economic Agent...")
        state = await macro_agent.process(state)

        # Agent 5: Sentiment Analysis
        print("\n🔄 Running Sentiment Agent...")
        state = await sentiment_agent.process(state)

        # =================================================================
        # PHASE 3: STRATEGY AND PLANNING
        # =================================================================
        print("\n🧠 PHASE 3: STRATEGY AND PLANNING")
        print("-" * 50)

        # Agent 6: AI Strategist (Uses all previous agent data)
        print("\n🔄 Running AI Strategist Agent...")
        state = await strategist_agent.process(state)

        # Agent 7: Financial Planner (Optional - if goal provided)
        if financial_goal:
            print("\n🔄 Running Financial Planner Agent...")
            state = await planner_agent.process(state, financial_goal)
        else:
            print("\n⏭️  Skipping Financial Planner (no goal provided)")

        # =================================================================
        # PHASE 4: RESULTS SUMMARY
        # =================================================================
        print("\n📈 PHASE 4: PIPELINE RESULTS SUMMARY")
        print("=" * 60)

        print_pipeline_summary(state)

        print("\n🎉 PIPELINE COMPLETED SUCCESSFULLY!")
        print("=" * 60)

        return state

    except Exception as e:
        print(f"\n❌ PIPELINE ERROR: {e}")
        print("=" * 60)
        return state

def print_pipeline_summary(state):
    """Print a comprehensive summary of all pipeline results"""

    symbol = state.get('symbol', 'UNKNOWN')
    print(f"\n📊 ANALYSIS SUMMARY FOR {symbol}")
    print("-" * 40)

    # Market Data Summary
    if 'market_data' in state and state['market_data']:
        md = state['market_data']
        print(f"\n💰 MARKET DATA:")
        print(f"   Current Price: ${md.current_price:.2f}")
        print(f"   Trend: {md.trend}")
        print(f"   RSI: {md.rsi:.1f}")
        print(f"   1-Day Return: {md.return_1d:.2%}")
        print(f"   20-Day Volatility: {md.volatility_20d:.1%}")
        print(f"   Support Level: ${md.support_level:.2f}")
        print(f"   Resistance Level: ${md.resistance_level:.2f}")

    # Risk Metrics Summary
    if 'risk_metrics' in state and state['risk_metrics']:
        rm = state['risk_metrics']
        print(f"\n⚠️  RISK ANALYSIS:")
        print(f"   Portfolio Volatility: {rm.portfolio_volatility:.1%}")
        print(f"   Sharpe Ratio: {rm.sharpe_ratio:.2f}")
        print(f"   Maximum Drawdown: {rm.maximum_drawdown:.1%}")
        print(f"   VaR (5%): {rm.value_at_risk_5pct:.1%}")
        print(f"   Expected Shortfall: {rm.expected_shortfall:.1%}")
        print(f"   GARCH Volatility: {rm.garch_volatility:.1%}")

    # Forecast Summary
    if 'forecast_data' in state and state['forecast_data']:
        fd = state['forecast_data']
        current_price = state.get('market_data', {}).current_price if 'market_data' in state else 100
        price_change = ((fd.ensemble_forecast - current_price) / current_price) * 100

        print(f"\n🔮 FORECASTING:")
        print(f"   Ensemble Forecast: ${fd.ensemble_forecast:.2f} ({price_change:+.1f}%)")
        print(f"   Forecast Confidence: {fd.forecast_confidence:.1%}")
        print(f"   Upside Probability: {fd.upside_probability:.1%}")
        print(f"   Downside Risk: {fd.downside_risk:.1%}")
        print(f"   Volatility Forecast: {fd.volatility_forecast:.1%}")
        print(f"   Prediction Interval: ${fd.prediction_interval[0]:.2f} - ${fd.prediction_interval[1]:.2f}")

    # Macro Economic Summary
    if 'macro_data' in state and state['macro_data']:
        macro = state['macro_data']
        print(f"\n🌍 MACRO ECONOMICS:")
        print(f"   Market Sentiment: {macro.market_sentiment}")
        print(f"   GDP Growth: {macro.gdp_growth:.1f}%")
        print(f"   Inflation Rate: {macro.inflation_rate:.1f}%")
        print(f"   Fed Funds Rate: {macro.federal_funds_rate:.2f}%")
        print(f"   VIX: {macro.vix:.1f}")
        print(f"   Yield Curve Slope: {macro.yield_curve_slope:.2f}")

    # Sentiment Summary
    if 'sentiment_data' in state and state['sentiment_data']:
        sentiment = state['sentiment_data']
        print(f"\n💭 SENTIMENT ANALYSIS:")
        print(f"   Overall Sentiment: {sentiment.sentiment_trend} ({sentiment.overall_sentiment:.2f})")
        print(f"   News Sentiment: {sentiment.news_sentiment:.2f}")
        print(f"   Social Media Sentiment: {sentiment.social_media_sentiment:.2f}")
        print(f"   Fear & Greed Index: {sentiment.fear_greed_index:.0f}")
        print(f"   Analyst Rating Trend: {sentiment.analyst_rating_trend}")
        print(f"   Key Topics: {', '.join(sentiment.key_topics[:3])}")

    # AI Recommendation Summary
    if 'recommendation' in state and state['recommendation']:
        rec = state['recommendation']
        print(f"\n🧠 AI RECOMMENDATION:")
        print(f"   Action: {rec.action}")
        print(f"   Confidence: {rec.confidence:.1%}")
        print(f"   Position Size: {rec.position_size:.1%}")
        print(f"   Risk Level: {rec.risk_level}")
        print(f"   Time Horizon: {rec.time_horizon}")
        print(f"   Risk/Reward Ratio: {rec.risk_reward_ratio:.2f}")
        print(f"   Probability of Success: {rec.probability_of_success:.1%}")
        print(f"   Entry Price: ${rec.entry_price:.2f}")
        print(f"   Stop Loss: ${rec.stop_loss:.2f}")
        print(f"   Take Profit: ${rec.take_profit:.2f}")

        # Print key reasoning points
        if hasattr(rec, 'detailed_reasoning') and rec.detailed_reasoning:
            print(f"\n   💡 Key Reasoning:")
            reasoning_lines = rec.detailed_reasoning.split('\n')[:3]  # First 3 lines
            for line in reasoning_lines:
                if line.strip():
                    print(f"      {line.strip()}")

    # Financial Plan Summary
    if 'financial_plan' in state and state['financial_plan']:
        plan = state['financial_plan']
        print(f"\n💰 FINANCIAL PLANNING:")
        print(f"   Goal Achievement Probability: {plan.success_probability:.1%}")
        print(f"   Projected Value: ${plan.projected_value:,.0f}")
        print(f"   Target Amount: ${plan.goal.target_amount:,.0f}")
        print(f"   Plan Sharpe Ratio: {plan.plan_sharpe_ratio:.2f}")
        print(f"   Expected Volatility: {plan.plan_volatility:.1%}")
        print(f"   Estimated Max Drawdown: {plan.plan_max_drawdown:.1%}")

        # Asset allocation
        print(f"\n   📊 Asset Allocation:")
        for asset, allocation in plan.asset_allocation.items():
            print(f"      {asset.replace('_', ' ').title()}: {allocation:.1%}")

        # Top recommendations
        print(f"\n   💡 Top Recommendations:")
        for i, rec in enumerate(plan.recommendations[:3], 1):
            print(f"      {i}. {rec}")

# =============================================================================
# VISUALIZATION FUNCTIONS
# =============================================================================

def create_dashboard_visualizations(state):
    """Create comprehensive visualizations for the pipeline results"""

    if not state or 'market_data' not in state:
        print("⚠️  No market data available for visualization")
        return

    # Create subplots for comprehensive dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Price Chart with Technical Indicators',
            'Risk Metrics Dashboard',
            'Forecast Comparison',
            'Sentiment Analysis',
            'Portfolio Allocation',
            'Monte Carlo Results'
        ],
        specs=[
            [{"secondary_y": True}, {"type": "indicator"}],
            [{"type": "bar"}, {"type": "pie"}],
            [{"type": "scatter"}, {"type": "histogram"}]
        ]
    )

    market_data = state['market_data']
    prices = market_data.prices

    # 1. Price chart with indicators
    fig.add_trace(
        go.Scatter(
            x=prices.index,
            y=prices.values,
            name="Price",
            line=dict(color='blue', width=2)
        ),
        row=1, col=1
    )

    # Add support and resistance levels
    fig.add_hline(
        y=market_data.support_level,
        line_dash="dash",
        line_color="green",
        annotation_text="Support",
        row=1, col=1
    )
    fig.add_hline(
        y=market_data.resistance_level,
        line_dash="dash",
        line_color="red",
        annotation_text="Resistance",
        row=1, col=1
    )

    # 2. Risk metrics gauge
    if 'risk_metrics' in state:
        rm = state['risk_metrics']
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=rm.sharpe_ratio,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "Sharpe Ratio"},
                gauge={
                    'axis': {'range': [-2, 3]},
                    'bar': {'color': "darkblue"},
                    'steps': [
                        {'range': [-2, 0], 'color': "lightgray"},
                        {'range': [0, 1], 'color': "yellow"},
                        {'range': [1, 3], 'color': "green"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 2.0
                    }
                }
            ),
            row=1, col=2
        )

    # 3. Forecast comparison
    if 'forecast_data' in state:
        fd = state['forecast_data']
        forecasts = ['ARIMA', 'Prophet', 'LSTM', 'Ensemble']
        values = [fd.arima_forecast, fd.prophet_forecast, fd.lstm_forecast, fd.ensemble_forecast]

        fig.add_trace(
            go.Bar(x=forecasts, y=values, name="Forecasts"),
            row=2, col=1
        )

    # 4. Sentiment pie chart
    if 'sentiment_data' in state:
        sentiment = state['sentiment_data']
        labels = ['Positive', 'Neutral', 'Negative']

        # Convert overall sentiment to distribution
        overall = sentiment.overall_sentiment
        if overall > 0.1:
            values = [60 + overall*20, 30, 10 - overall*10]
        elif overall < -0.1:
            values = [10 + overall*10, 30, 60 - overall*20]
        else:
            values = [40, 40, 20]

        fig.add_trace(
            go.Pie(labels=labels, values=values, name="Sentiment"),
            row=2, col=2
        )

    # 5. Portfolio allocation
    if 'financial_plan' in state:
        plan = state['financial_plan']
        assets = list(plan.asset_allocation.keys())
        allocations = list(plan.asset_allocation.values())

        fig.add_trace(
            go.Bar(
                x=assets,
                y=[a*100 for a in allocations],
                name="Allocation %"
            ),
            row=3, col=1
        )

    # 6. Monte Carlo histogram
    if 'financial_plan' in state and 'monte_carlo_results' in state['financial_plan']:
        mc = state['financial_plan'].monte_carlo_results
        percentiles = ['5th', '25th', '50th', '75th', '95th']
        values = [
            mc['percentile_5'], mc['percentile_25'], mc['percentile_50'],
            mc['percentile_75'], mc['percentile_95']
        ]

        fig.add_trace(
            go.Scatter(
                x=percentiles,
                y=values,
                mode='lines+markers',
                name="Monte Carlo Percentiles"
            ),
            row=3, col=2
        )

    # Update layout
    fig.update_layout(
        height=900,
        title_text="AI Financial Forecasting Dashboard",
        showlegend=True
    )

    # Show the dashboard
    fig.show()

    print("📊 Dashboard visualization created successfully!")

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

async def main_demo():
    """Main demo function showcasing the complete pipeline"""

    print("🎯 AI FINANCIAL FORECASTING SYSTEM - CAPSTONE DEMO")
    print("=" * 70)

    # Example 1: Basic stock analysis
    print("\n📈 EXAMPLE 1: BASIC STOCK ANALYSIS")
    print("-" * 50)

    state = await run_pipeline(
        symbol="AAPL",
        openai_api_key=None  # Set your OpenAI API key here for enhanced recommendations
    )

    # Example 2: Stock analysis with financial planning
    print("\n\n💰 EXAMPLE 2: STOCK ANALYSIS WITH FINANCIAL PLANNING")
    print("-" * 50)

    # Create a sample financial goal
    retirement_goal = FinancialGoal(
        target_amount=1000000,    # $1M retirement goal
        current_amount=50000,     # $50K current savings
        monthly_contribution=2000, # $2K monthly contributions
        time_horizon_years=25,    # 25 years to retirement
        risk_tolerance="moderate", # Moderate risk tolerance
        age=35,                   # 35 years old
        annual_income=120000,     # $120K annual income
        goal_type="retirement"
    )

    state_with_planning = await run_pipeline(
        symbol="TSLA",
        openai_api_key=None,  # Set your OpenAI API key here
        financial_goal=retirement_goal
    )

    # Create visualizations
    print("\n📊 CREATING DASHBOARD VISUALIZATIONS...")
    create_dashboard_visualizations(state_with_planning)

    print("\n🎉 DEMO COMPLETED SUCCESSFULLY!")
    print("=" * 70)

    return state_with_planning

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def save_pipeline_results(state, filename="pipeline_results.json"):
    """Save pipeline results to JSON file for later analysis"""

    # Convert state to JSON-serializable format
    serializable_state = {}

    for key, value in state.items():
        if hasattr(value, '__dict__'):
            # Convert dataclass to dict
            serializable_state[key] = {
                'type': value.__class__.__name__,
                'data': {k: v for k, v in value.__dict__.items()
                        if not isinstance(v, (pd.Series, pd.DataFrame))}
            }
        else:
            serializable_state[key] = value

    # Save to file
    with open(filename, 'w') as f:
        json.dump(serializable_state, f, indent=2, default=str)

    print(f"💾 Pipeline results saved to {filename}")

def load_pipeline_results(filename="pipeline_results.json"):
    """Load previously saved pipeline results"""

    with open(filename, 'r') as f:
        return json.load(f)

# =============================================================================
# RUN THE DEMO
# =============================================================================

# Notebook-compatible execution functions
def run_complete_analysis(symbol="AAPL", openai_api_key=None):
    """
    Notebook-compatible wrapper to run the complete pipeline

    Usage:
        # Basic analysis
        results = run_complete_analysis("AAPL")

        # With financial planning
        goal = FinancialGoal(target_amount=1000000, current_amount=50000,
                           monthly_contribution=2000, time_horizon_years=25,
                           risk_tolerance="moderate", age=35)
        results = run_complete_analysis("TSLA", goal=goal)
    """
    try:
        # Try to get existing event loop (for Jupyter/Colab)
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in a notebook - use nest_asyncio or create task
            import nest_asyncio
            nest_asyncio.apply()
            return asyncio.run(run_pipeline(symbol, openai_api_key))
        else:
            return asyncio.run(run_pipeline(symbol, openai_api_key))
    except ImportError:
        # nest_asyncio not available, try alternative approach
        try:
            loop = asyncio.get_event_loop()
            task = loop.create_task(run_pipeline(symbol, openai_api_key))
            return loop.run_until_complete(task)
        except RuntimeError:
            # Last resort - create new event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                return loop.run_until_complete(run_pipeline(symbol, openai_api_key))
            finally:
                loop.close()

def run_with_financial_planning(symbol="AAPL", target_amount=1000000,
                               current_amount=50000, monthly_contribution=2000,
                               time_horizon_years=25, age=35, risk_tolerance="moderate"):
    """
    Convenient function to run analysis with financial planning
    """
    goal = FinancialGoal(
        target_amount=target_amount,
        current_amount=current_amount,
        monthly_contribution=monthly_contribution,
        time_horizon_years=time_horizon_years,
        risk_tolerance=risk_tolerance,
        age=age,
        annual_income=100000,
        goal_type="retirement"
    )

    try:
        # Try to get existing event loop (for Jupyter/Colab)
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            return asyncio.run(run_pipeline(symbol, None, goal))
        else:
            return asyncio.run(run_pipeline(symbol, None, goal))
    except ImportError:
        # nest_asyncio not available, try alternative approach
        try:
            loop = asyncio.get_event_loop()
            task = loop.create_task(run_pipeline(symbol, None, goal))
            return loop.run_until_complete(task)
        except RuntimeError:
            # Last resort - create new event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                return loop.run_until_complete(run_pipeline(symbol, None, goal))
            finally:
                loop.close()

# Simplified synchronous version for notebooks
async def run_pipeline_sync(symbol="AAPL", openai_api_key=None, financial_goal=None):
    """Await this function directly in notebooks"""
    return await run_pipeline(symbol, openai_api_key, financial_goal)

if __name__ == "__main__":
    # Check if we're in a notebook environment
    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            print("📓 Detected notebook environment")
            print("🚀 Running quick AAPL analysis...")

            # For notebooks, use await syntax instead
            print("Use this code in a notebook cell:")
            print("```python")
            print("# Basic analysis")
            print("results = await run_pipeline_sync('AAPL')")
            print("")
            print("# With financial planning")
            print("goal = FinancialGoal(target_amount=1000000, current_amount=50000,")
            print("                    monthly_contribution=2000, time_horizon_years=25,")
            print("                    risk_tolerance='moderate', age=35)")
            print("results = await run_pipeline_sync('TSLA', financial_goal=goal)")
            print("```")

        else:
            # Regular Python environment
            print("🚀 Running quick AAPL analysis...")
            results = run_complete_analysis("AAPL")

            print("\n🚀 Running TSLA analysis with retirement planning...")
            results_with_planning = run_with_financial_planning(
                symbol="TSLA",
                target_amount=2000000,
                current_amount=100000,
                monthly_contribution=3000,
                time_horizon_years=20,
                age=40,
                risk_tolerance="aggressive"
            )

            save_pipeline_results(results_with_planning, "tsla_retirement_analysis.json")
            print("\n✅ Analysis complete!")

    except ImportError:
        # No IPython, assume regular Python
        print("🚀 Running quick AAPL analysis...")
        results = run_complete_analysis("AAPL")

        print("\n🚀 Running TSLA analysis with retirement planning...")
        results_with_planning = run_with_financial_planning(
            symbol="TSLA",
            target_amount=2000000,
            current_amount=100000,
            monthly_contribution=3000,
            time_horizon_years=20,
            age=40,
            risk_tolerance="aggressive"
        )

        save_pipeline_results(results_with_planning, "tsla_retirement_analysis.json")
        print("\n✅ Analysis complete!")

📓 Detected notebook environment
🚀 Running quick AAPL analysis...
Use this code in a notebook cell:
```python
# Basic analysis
results = await run_pipeline_sync('AAPL')

# With financial planning
goal = FinancialGoal(target_amount=1000000, current_amount=50000,
                    monthly_contribution=2000, time_horizon_years=25,
                    risk_tolerance='moderate', age=35)
results = await run_pipeline_sync('TSLA', financial_goal=goal)
```


In [23]:
# CAPSTONE PROJECT: AI FINANCIAL FORECASTING SYSTEM
# Part 9: Pipeline Orchestration and Main Execution

# =============================================================================
# PIPELINE ORCHESTRATION
# =============================================================================

async def run_pipeline(symbol="AAPL", openai_api_key=None, financial_goal=None):
    """
    Run the complete AI Financial Forecasting pipeline with all agents

    Args:
        symbol: Stock symbol to analyze (default: "AAPL")
        openai_api_key: OpenAI API key for enhanced recommendations (optional)
        financial_goal: FinancialGoal object for planning (optional)

    Returns:
        dict: Complete state with all agent results
    """

    print("🚀 STARTING AI FINANCIAL FORECASTING PIPELINE")
    print("=" * 60)

    # Initialize empty state
    state = {}

    # Initialize all agents
    print("\n📋 INITIALIZING AGENTS...")
    market_agent = MarketDataAgent()
    risk_agent = RiskAgent()
    forecast_agent = ForecastingAgent()
    macro_agent = MacroEconomicAgent()
    sentiment_agent = SentimentAgent()
    strategist_agent = StrategistAgent(api_key=openai_api_key)
    planner_agent = FinancialPlannerAgent()

    print(f"✅ All agents initialized successfully\n")

    try:
        # =================================================================
        # PHASE 1: DATA COLLECTION AND ANALYSIS
        # =================================================================
        print("📊 PHASE 1: DATA COLLECTION AND ANALYSIS")
        print("-" * 50)

        # Agent 1: Market Data (Foundation - required by others)
        print("\n🔄 Running Market Data Agent...")
        state = await market_agent.process(state, symbol=symbol, period="1y")

        # Agent 2: Risk Analysis (Uses market data)
        print("\n🔄 Running Risk Agent...")
        state = await risk_agent.process(state)

        # Agent 3: Forecasting (Uses market data)
        print("\n🔄 Running Forecasting Agent...")
        state = await forecast_agent.process(state, forecast_horizon=5)

        # =================================================================
        # PHASE 2: MACRO AND SENTIMENT ANALYSIS
        # =================================================================
        print("\n🌍 PHASE 2: MACRO AND SENTIMENT ANALYSIS")
        print("-" * 50)

        # Agent 4: Macro Economic Analysis
        print("\n🔄 Running Macro Economic Agent...")
        state = await macro_agent.process(state)

        # Agent 5: Sentiment Analysis
        print("\n🔄 Running Sentiment Agent...")
        state = await sentiment_agent.process(state)

        # =================================================================
        # PHASE 3: STRATEGY AND PLANNING
        # =================================================================
        print("\n🧠 PHASE 3: STRATEGY AND PLANNING")
        print("-" * 50)

        # Agent 6: AI Strategist (Uses all previous agent data)
        print("\n🔄 Running AI Strategist Agent...")
        state = await strategist_agent.process(state)

        # Agent 7: Financial Planner (Optional - if goal provided)
        if financial_goal:
            print("\n🔄 Running Financial Planner Agent...")
            state = await planner_agent.process(state, financial_goal)
        else:
            print("\n⏭️  Skipping Financial Planner (no goal provided)")

        # =================================================================
        # PHASE 4: RESULTS SUMMARY
        # =================================================================
        print("\n📈 PHASE 4: PIPELINE RESULTS SUMMARY")
        print("=" * 60)

        print_pipeline_summary(state)

        print("\n🎉 PIPELINE COMPLETED SUCCESSFULLY!")
        print("=" * 60)

        return state

    except Exception as e:
        print(f"\n❌ PIPELINE ERROR: {e}")
        print("=" * 60)
        return state

def print_pipeline_summary(state):
    """Print a comprehensive summary of all pipeline results"""

    symbol = state.get('symbol', 'UNKNOWN')
    print(f"\n📊 ANALYSIS SUMMARY FOR {symbol}")
    print("-" * 40)

    # Market Data Summary
    if 'market_data' in state and state['market_data']:
        md = state['market_data']
        print(f"\n💰 MARKET DATA:")
        print(f"   Current Price: ${md.current_price:.2f}")
        print(f"   Trend: {md.trend}")
        print(f"   RSI: {md.rsi:.1f}")
        print(f"   1-Day Return: {md.return_1d:.2%}")
        print(f"   20-Day Volatility: {md.volatility_20d:.1%}")
        print(f"   Support Level: ${md.support_level:.2f}")
        print(f"   Resistance Level: ${md.resistance_level:.2f}")

    # Risk Metrics Summary
    if 'risk_metrics' in state and state['risk_metrics']:
        rm = state['risk_metrics']
        print(f"\n⚠️  RISK ANALYSIS:")
        print(f"   Portfolio Volatility: {rm.portfolio_volatility:.1%}")
        print(f"   Sharpe Ratio: {rm.sharpe_ratio:.2f}")
        print(f"   Maximum Drawdown: {rm.maximum_drawdown:.1%}")
        print(f"   VaR (5%): {rm.value_at_risk_5pct:.1%}")
        print(f"   Expected Shortfall: {rm.expected_shortfall:.1%}")
        print(f"   GARCH Volatility: {rm.garch_volatility:.1%}")

    # Forecast Summary
    if 'forecast_data' in state and state['forecast_data']:
        fd = state['forecast_data']
        current_price = state.get('market_data', {}).current_price if 'market_data' in state else 100
        price_change = ((fd.ensemble_forecast - current_price) / current_price) * 100

        print(f"\n🔮 FORECASTING:")
        print(f"   Ensemble Forecast: ${fd.ensemble_forecast:.2f} ({price_change:+.1f}%)")
        print(f"   Forecast Confidence: {fd.forecast_confidence:.1%}")
        print(f"   Upside Probability: {fd.upside_probability:.1%}")
        print(f"   Downside Risk: {fd.downside_risk:.1%}")
        print(f"   Volatility Forecast: {fd.volatility_forecast:.1%}")
        print(f"   Prediction Interval: ${fd.prediction_interval[0]:.2f} - ${fd.prediction_interval[1]:.2f}")

    # Macro Economic Summary
    if 'macro_data' in state and state['macro_data']:
        macro = state['macro_data']
        print(f"\n🌍 MACRO ECONOMICS:")
        print(f"   Market Sentiment: {macro.market_sentiment}")
        print(f"   GDP Growth: {macro.gdp_growth:.1f}%")
        print(f"   Inflation Rate: {macro.inflation_rate:.1f}%")
        print(f"   Fed Funds Rate: {macro.federal_funds_rate:.2f}%")
        print(f"   VIX: {macro.vix:.1f}")
        print(f"   Yield Curve Slope: {macro.yield_curve_slope:.2f}")

    # Sentiment Summary
    if 'sentiment_data' in state and state['sentiment_data']:
        sentiment = state['sentiment_data']
        print(f"\n💭 SENTIMENT ANALYSIS:")
        print(f"   Overall Sentiment: {sentiment.sentiment_trend} ({sentiment.overall_sentiment:.2f})")
        print(f"   News Sentiment: {sentiment.news_sentiment:.2f}")
        print(f"   Social Media Sentiment: {sentiment.social_media_sentiment:.2f}")
        print(f"   Fear & Greed Index: {sentiment.fear_greed_index:.0f}")
        print(f"   Analyst Rating Trend: {sentiment.analyst_rating_trend}")
        print(f"   Key Topics: {', '.join(sentiment.key_topics[:3])}")

    # AI Recommendation Summary
    if 'recommendation' in state and state['recommendation']:
        rec = state['recommendation']
        print(f"\n🧠 AI RECOMMENDATION:")
        print(f"   Action: {rec.action}")
        print(f"   Confidence: {rec.confidence:.1%}")
        print(f"   Position Size: {rec.position_size:.1%}")
        print(f"   Risk Level: {rec.risk_level}")
        print(f"   Time Horizon: {rec.time_horizon}")
        print(f"   Risk/Reward Ratio: {rec.risk_reward_ratio:.2f}")
        print(f"   Probability of Success: {rec.probability_of_success:.1%}")
        print(f"   Entry Price: ${rec.entry_price:.2f}")
        print(f"   Stop Loss: ${rec.stop_loss:.2f}")
        print(f"   Take Profit: ${rec.take_profit:.2f}")

        # Print key reasoning points
        if hasattr(rec, 'detailed_reasoning') and rec.detailed_reasoning:
            print(f"\n   💡 Key Reasoning:")
            reasoning_lines = rec.detailed_reasoning.split('\n')[:3]  # First 3 lines
            for line in reasoning_lines:
                if line.strip():
                    print(f"      {line.strip()}")

    # Financial Plan Summary
    if 'financial_plan' in state and state['financial_plan']:
        plan = state['financial_plan']
        print(f"\n💰 FINANCIAL PLANNING:")
        print(f"   Goal Achievement Probability: {plan.success_probability:.1%}")
        print(f"   Projected Value: ${plan.projected_value:,.0f}")
        print(f"   Target Amount: ${plan.goal.target_amount:,.0f}")
        print(f"   Plan Sharpe Ratio: {plan.plan_sharpe_ratio:.2f}")
        print(f"   Expected Volatility: {plan.plan_volatility:.1%}")
        print(f"   Estimated Max Drawdown: {plan.plan_max_drawdown:.1%}")

        # Asset allocation
        print(f"\n   📊 Asset Allocation:")
        for asset, allocation in plan.asset_allocation.items():
            print(f"      {asset.replace('_', ' ').title()}: {allocation:.1%}")

        # Top recommendations
        print(f"\n   💡 Top Recommendations:")
        for i, rec in enumerate(plan.recommendations[:3], 1):
            print(f"      {i}. {rec}")

# =============================================================================
# VISUALIZATION FUNCTIONS
# =============================================================================

def create_dashboard_visualizations(state):
    """Create comprehensive visualizations for the pipeline results"""

    if not state or 'market_data' not in state:
        print("⚠️  No market data available for visualization")
        return

    # Create subplots for comprehensive dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Price Chart with Technical Indicators',
            'Risk Metrics Dashboard',
            'Forecast Comparison',
            'Sentiment Analysis',
            'Portfolio Allocation',
            'Monte Carlo Results'
        ],
        specs=[
            [{"secondary_y": True}, {"type": "indicator"}],
            [{"type": "bar"}, {"type": "pie"}],
            [{"type": "scatter"}, {"type": "histogram"}]
        ]
    )

    market_data = state['market_data']
    prices = market_data.prices

    # 1. Price chart with indicators
    fig.add_trace(
        go.Scatter(
            x=prices.index,
            y=prices.values,
            name="Price",
            line=dict(color='blue', width=2)
        ),
        row=1, col=1
    )

    # Add support and resistance levels
    fig.add_hline(
        y=market_data.support_level,
        line_dash="dash",
        line_color="green",
        annotation_text="Support",
        row=1, col=1
    )
    fig.add_hline(
        y=market_data.resistance_level,
        line_dash="dash",
        line_color="red",
        annotation_text="Resistance",
        row=1, col=1
    )

    # 2. Risk metrics gauge
    if 'risk_metrics' in state:
        rm = state['risk_metrics']
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=rm.sharpe_ratio,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "Sharpe Ratio"},
                gauge={
                    'axis': {'range': [-2, 3]},
                    'bar': {'color': "darkblue"},
                    'steps': [
                        {'range': [-2, 0], 'color': "lightgray"},
                        {'range': [0, 1], 'color': "yellow"},
                        {'range': [1, 3], 'color': "green"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 2.0
                    }
                }
            ),
            row=1, col=2
        )

    # 3. Forecast comparison
    if 'forecast_data' in state:
        fd = state['forecast_data']
        forecasts = ['ARIMA', 'Prophet', 'LSTM', 'Ensemble']
        values = [fd.arima_forecast, fd.prophet_forecast, fd.lstm_forecast, fd.ensemble_forecast]

        fig.add_trace(
            go.Bar(x=forecasts, y=values, name="Forecasts"),
            row=2, col=1
        )

    # 4. Sentiment pie chart
    if 'sentiment_data' in state:
        sentiment = state['sentiment_data']
        labels = ['Positive', 'Neutral', 'Negative']

        # Convert overall sentiment to distribution
        overall = sentiment.overall_sentiment
        if overall > 0.1:
            values = [60 + overall*20, 30, 10 - overall*10]
        elif overall < -0.1:
            values = [10 + overall*10, 30, 60 - overall*20]
        else:
            values = [40, 40, 20]

        fig.add_trace(
            go.Pie(labels=labels, values=values, name="Sentiment"),
            row=2, col=2
        )

    # 5. Portfolio allocation
    if 'financial_plan' in state:
        plan = state['financial_plan']
        assets = list(plan.asset_allocation.keys())
        allocations = list(plan.asset_allocation.values())

        fig.add_trace(
            go.Bar(
                x=assets,
                y=[a*100 for a in allocations],
                name="Allocation %"
            ),
            row=3, col=1
        )

    # 6. Monte Carlo histogram
    if 'financial_plan' in state and 'monte_carlo_results' in state['financial_plan']:
        mc = state['financial_plan'].monte_carlo_results
        percentiles = ['5th', '25th', '50th', '75th', '95th']
        values = [
            mc['percentile_5'], mc['percentile_25'], mc['percentile_50'],
            mc['percentile_75'], mc['percentile_95']
        ]

        fig.add_trace(
            go.Scatter(
                x=percentiles,
                y=values,
                mode='lines+markers',
                name="Monte Carlo Percentiles"
            ),
            row=3, col=2
        )

    # Update layout
    fig.update_layout(
        height=900,
        title_text="AI Financial Forecasting Dashboard",
        showlegend=True
    )

    # Show the dashboard
    fig.show()

    print("📊 Dashboard visualization created successfully!")

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

async def main_demo():
    """Main demo function showcasing the complete pipeline"""

    print("🎯 AI FINANCIAL FORECASTING SYSTEM - CAPSTONE DEMO")
    print("=" * 70)

    # Example 1: Basic stock analysis
    print("\n📈 EXAMPLE 1: BASIC STOCK ANALYSIS")
    print("-" * 50)

    state = await run_pipeline(
        symbol="AAPL",
        openai_api_key=None  # Set your OpenAI API key here for enhanced recommendations
    )

    # Example 2: Stock analysis with financial planning
    print("\n\n💰 EXAMPLE 2: STOCK ANALYSIS WITH FINANCIAL PLANNING")
    print("-" * 50)

    # Create a sample financial goal
    retirement_goal = FinancialGoal(
        target_amount=1000000,    # $1M retirement goal
        current_amount=50000,     # $50K current savings
        monthly_contribution=2000, # $2K monthly contributions
        time_horizon_years=25,    # 25 years to retirement
        risk_tolerance="moderate", # Moderate risk tolerance
        age=35,                   # 35 years old
        annual_income=120000,     # $120K annual income
        goal_type="retirement"
    )

    state_with_planning = await run_pipeline(
        symbol="TSLA",
        openai_api_key=None,  # Set your OpenAI API key here
        financial_goal=retirement_goal
    )

    # Create visualizations
    print("\n📊 CREATING DASHBOARD VISUALIZATIONS...")
    create_dashboard_visualizations(state_with_planning)

    print("\n🎉 DEMO COMPLETED SUCCESSFULLY!")
    print("=" * 70)

    return state_with_planning

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def save_pipeline_results(state, filename="pipeline_results.json"):
    """Save pipeline results to JSON file for later analysis"""

    # Convert state to JSON-serializable format
    serializable_state = {}

    for key, value in state.items():
        if hasattr(value, '__dict__'):
            # Convert dataclass to dict
            serializable_state[key] = {
                'type': value.__class__.__name__,
                'data': {k: v for k, v in value.__dict__.items()
                        if not isinstance(v, (pd.Series, pd.DataFrame))}
            }
        else:
            serializable_state[key] = value

    # Save to file
    with open(filename, 'w') as f:
        json.dump(serializable_state, f, indent=2, default=str)

    print(f"💾 Pipeline results saved to {filename}")

def load_pipeline_results(filename="pipeline_results.json"):
    """Load previously saved pipeline results"""

    with open(filename, 'r') as f:
        return json.load(f)

# =============================================================================
# RUN THE DEMO
# =============================================================================

# Notebook-compatible execution functions
def run_complete_analysis(symbol="AAPL", openai_api_key=None):
    """
    Notebook-compatible wrapper to run the complete pipeline

    Usage:
        # Basic analysis
        results = run_complete_analysis("AAPL")

        # With financial planning
        goal = FinancialGoal(target_amount=1000000, current_amount=50000,
                           monthly_contribution=2000, time_horizon_years=25,
                           risk_tolerance="moderate", age=35)
        results = run_complete_analysis("TSLA", goal=goal)
    """
    try:
        # Try to get existing event loop (for Jupyter/Colab)
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in a notebook - use nest_asyncio or create task
            import nest_asyncio
            nest_asyncio.apply()
            return asyncio.run(run_pipeline(symbol, openai_api_key))
        else:
            return asyncio.run(run_pipeline(symbol, openai_api_key))
    except ImportError:
        # nest_asyncio not available, try alternative approach
        try:
            loop = asyncio.get_event_loop()
            task = loop.create_task(run_pipeline(symbol, openai_api_key))
            return loop.run_until_complete(task)
        except RuntimeError:
            # Last resort - create new event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                return loop.run_until_complete(run_pipeline(symbol, openai_api_key))
            finally:
                loop.close()

def run_with_financial_planning(symbol="AAPL", target_amount=1000000,
                               current_amount=50000, monthly_contribution=2000,
                               time_horizon_years=25, age=35, risk_tolerance="moderate"):
    """
    Convenient function to run analysis with financial planning
    """
    goal = FinancialGoal(
        target_amount=target_amount,
        current_amount=current_amount,
        monthly_contribution=monthly_contribution,
        time_horizon_years=time_horizon_years,
        risk_tolerance=risk_tolerance,
        age=age,
        annual_income=100000,
        goal_type="retirement"
    )

    try:
        # Try to get existing event loop (for Jupyter/Colab)
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            return asyncio.run(run_pipeline(symbol, None, goal))
        else:
            return asyncio.run(run_pipeline(symbol, None, goal))
    except ImportError:
        # nest_asyncio not available, try alternative approach
        try:
            loop = asyncio.get_event_loop()
            task = loop.create_task(run_pipeline(symbol, None, goal))
            return loop.run_until_complete(task)
        except RuntimeError:
            # Last resort - create new event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                return loop.run_until_complete(run_pipeline(symbol, None, goal))
            finally:
                loop.close()

# Simplified synchronous version for notebooks
async def run_pipeline_sync(symbol="AAPL", openai_api_key=None, financial_goal=None):
    """Await this function directly in notebooks"""
    return await run_pipeline(symbol, openai_api_key, financial_goal)

if __name__ == "__main__":
    # Check if we're in a notebook environment
    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            print("📓 Detected notebook environment")
            print("🚀 Running quick AAPL analysis...")

            # For notebooks, use await syntax instead
            print("Use this code in a notebook cell:")
            print("```python")
            print("# Basic analysis")
            print("results = await run_pipeline_sync('AAPL')")
            print("")
            print("# With financial planning")
            print("goal = FinancialGoal(target_amount=1000000, current_amount=50000,")
            print("                    monthly_contribution=2000, time_horizon_years=25,")
            print("                    risk_tolerance='moderate', age=35)")
            print("results = await run_pipeline_sync('TSLA', financial_goal=goal)")
            print("```")

        else:
            # Regular Python environment
            print("🚀 Running quick AAPL analysis...")
            results = run_complete_analysis("AAPL")

            print("\n🚀 Running TSLA analysis with retirement planning...")
            results_with_planning = run_with_financial_planning(
                symbol="TSLA",
                target_amount=2000000,
                current_amount=100000,
                monthly_contribution=3000,
                time_horizon_years=20,
                age=40,
                risk_tolerance="aggressive"
            )

            save_pipeline_results(results_with_planning, "tsla_retirement_analysis.json")
            print("\n✅ Analysis complete!")

    except ImportError:
        # No IPython, assume regular Python
        print("🚀 Running quick AAPL analysis...")
        results = run_complete_analysis("AAPL")

        print("\n🚀 Running TSLA analysis with retirement planning...")
        results_with_planning = run_with_financial_planning(
            symbol="TSLA",
            target_amount=2000000,
            current_amount=100000,
            monthly_contribution=3000,
            time_horizon_years=20,
            age=40,
            risk_tolerance="aggressive"
        )

        save_pipeline_results(results_with_planning, "tsla_retirement_analysis.json")
        print("\n✅ Analysis complete!")

📓 Detected notebook environment
🚀 Running quick AAPL analysis...
Use this code in a notebook cell:
```python
# Basic analysis
results = await run_pipeline_sync('AAPL')

# With financial planning
goal = FinancialGoal(target_amount=1000000, current_amount=50000,
                    monthly_contribution=2000, time_horizon_years=25,
                    risk_tolerance='moderate', age=35)
results = await run_pipeline_sync('TSLA', financial_goal=goal)
```


In [24]:
# Basic AAPL analysis
results = await run_pipeline_sync('AAPL')

# With financial planning
goal = FinancialGoal(
    target_amount=1000000,
    current_amount=50000,
    monthly_contribution=2000,
    time_horizon_years=25,
    risk_tolerance='moderate',
    age=35,
    annual_income=120000,
    goal_type="retirement"
)
results = await run_pipeline_sync('TSLA', financial_goal=goal)

🚀 STARTING AI FINANCIAL FORECASTING PIPELINE

📋 INITIALIZING AGENTS...
✅ MarketDataAgent initialized
✅ RiskAgent initialized
✅ ForecastingAgent initialized
✅ MacroEconomicAgent initialized
✅ SentimentAgent initialized
✅ StrategistAgent initialized with rule-based logic
✅ FinancialPlannerAgent initialized
✅ All agents initialized successfully

📊 PHASE 1: DATA COLLECTION AND ANALYSIS
--------------------------------------------------

🔄 Running Market Data Agent...
📊 MarketDataAgent: Fetching market data for AAPL...
✅ MarketDataAgent: Loaded 250 data points
   Price: $230.56
   Trend: strongly_bullish
   RSI: 74.0

🔄 Running Risk Agent...
📊 RiskAgent: Computing portfolio risk metrics...
✅ RiskAgent: Risk analysis complete
   Volatility: 32.1%
   Sharpe Ratio: 0.17
   Max Drawdown: -33.4%
   VaR (5%): -3.1%
   GARCH Vol: 27.1%

🔄 Running Forecasting Agent...
🔮 ForecastingAgent: Generating forecasts...
✅ ForecastingAgent: Ensemble forecast: $229.57 (-0.4%)
   Confidence: 82.6%
   Upside Pr

In [25]:
# Run this once in your notebook
!pip install nest_asyncio

# Then you can use the regular functions
results = run_complete_analysis("AAPL")

🚀 STARTING AI FINANCIAL FORECASTING PIPELINE

📋 INITIALIZING AGENTS...
✅ MarketDataAgent initialized
✅ RiskAgent initialized
✅ ForecastingAgent initialized
✅ MacroEconomicAgent initialized
✅ SentimentAgent initialized
✅ StrategistAgent initialized with rule-based logic
✅ FinancialPlannerAgent initialized
✅ All agents initialized successfully

📊 PHASE 1: DATA COLLECTION AND ANALYSIS
--------------------------------------------------

🔄 Running Market Data Agent...
📊 MarketDataAgent: Fetching market data for AAPL...
✅ MarketDataAgent: Loaded 250 data points
   Price: $230.56
   Trend: strongly_bullish
   RSI: 74.0

🔄 Running Risk Agent...
📊 RiskAgent: Computing portfolio risk metrics...
✅ RiskAgent: Risk analysis complete
   Volatility: 32.1%
   Sharpe Ratio: 0.17
   Max Drawdown: -33.4%
   VaR (5%): -3.1%
   GARCH Vol: 27.1%

🔄 Running Forecasting Agent...
🔮 ForecastingAgent: Generating forecasts...
✅ ForecastingAgent: Ensemble forecast: $238.03 (+3.2%)
   Confidence: 82.2%
   Upside Pr

In [26]:
# This should work in your current environment
results = await run_pipeline_sync('AAPL')

🚀 STARTING AI FINANCIAL FORECASTING PIPELINE

📋 INITIALIZING AGENTS...
✅ MarketDataAgent initialized
✅ RiskAgent initialized
✅ ForecastingAgent initialized
✅ MacroEconomicAgent initialized
✅ SentimentAgent initialized
✅ StrategistAgent initialized with rule-based logic
✅ FinancialPlannerAgent initialized
✅ All agents initialized successfully

📊 PHASE 1: DATA COLLECTION AND ANALYSIS
--------------------------------------------------

🔄 Running Market Data Agent...
📊 MarketDataAgent: Fetching market data for AAPL...
✅ MarketDataAgent: Loaded 250 data points
   Price: $230.56
   Trend: strongly_bullish
   RSI: 74.0

🔄 Running Risk Agent...
📊 RiskAgent: Computing portfolio risk metrics...
✅ RiskAgent: Risk analysis complete
   Volatility: 32.1%
   Sharpe Ratio: 0.17
   Max Drawdown: -33.4%
   VaR (5%): -3.1%
   GARCH Vol: 27.1%

🔄 Running Forecasting Agent...
🔮 ForecastingAgent: Generating forecasts...
✅ ForecastingAgent: Ensemble forecast: $230.95 (+0.2%)
   Confidence: 83.3%
   Upside Pr

In [27]:
# Create a retirement goal
my_goal = FinancialGoal(
    target_amount=1000000,        # $1M retirement target
    current_amount=50000,         # Current savings
    monthly_contribution=2000,    # Monthly contributions
    time_horizon_years=25,        # Years to retirement
    risk_tolerance="moderate",    # Risk level
    age=35,                      # Your age
    annual_income=120000,        # Annual income
    goal_type="retirement"       # Goal type
)

# Run analysis with financial planning
results = await run_pipeline_sync('AAPL', financial_goal=my_goal)

🚀 STARTING AI FINANCIAL FORECASTING PIPELINE

📋 INITIALIZING AGENTS...
✅ MarketDataAgent initialized
✅ RiskAgent initialized
✅ ForecastingAgent initialized
✅ MacroEconomicAgent initialized
✅ SentimentAgent initialized
✅ StrategistAgent initialized with rule-based logic
✅ FinancialPlannerAgent initialized
✅ All agents initialized successfully

📊 PHASE 1: DATA COLLECTION AND ANALYSIS
--------------------------------------------------

🔄 Running Market Data Agent...
📊 MarketDataAgent: Fetching market data for AAPL...
✅ MarketDataAgent: Loaded 250 data points
   Price: $230.56
   Trend: strongly_bullish
   RSI: 74.0

🔄 Running Risk Agent...
📊 RiskAgent: Computing portfolio risk metrics...
✅ RiskAgent: Risk analysis complete
   Volatility: 32.1%
   Sharpe Ratio: 0.17
   Max Drawdown: -33.4%
   VaR (5%): -3.1%
   GARCH Vol: 27.1%

🔄 Running Forecasting Agent...
🔮 ForecastingAgent: Generating forecasts...
✅ ForecastingAgent: Ensemble forecast: $238.78 (+3.6%)
   Confidence: 81.9%
   Upside Pr